<a href="https://colab.research.google.com/github/ELehmann91/Thesis_Multilingual_Transferlearning/blob/master/Results_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train script
  
using prewritten fuctions and standardized data

In [167]:
import pandas as pd
from google.colab import drive, files

drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [168]:
%%capture
!pip install eli5
!git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'

%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import coicop_model
import model_helper
import pandas as pd
import numpy as np
from tqdm import tqdm
import io

## Import Data
  
data in the normalized folder are splitted between languages (currently de fr & it) and share the same columns so they can be merged easily

In [169]:
import pandas as pd
import numpy as np

data_path = '/data/'#
#file_path = 'fra/carrfour_trans_pred.csv'
file_path = 'normalized/norm_fr.csv'
file_path2 = 'normalized/norm_de.csv'
file_path3 = 'at/norm_at.csv'
file_path4 = 'normalized/df_italiano.csv'
#file_path = 'edeka_pred.csv'

# french data
df_fr = pd.read_csv(path+data_path+file_path,sep='|',index_col=False)

# only already labeled
df_fr = df_fr[df_fr['cc5'].isna()==False]
df_fr = df_fr[df_fr['shop'].isin(['carrefour','auchan','banque_de_france'])]#,'banque_de_france','carrefour'])]
df_fr['cc5'] = df_fr['cc5'].apply(lambda x: '9999_Non-Food' if int(str(x)[0])>2 else x)
print(len(df_fr))

# german data
df_de = pd.read_csv(path+data_path+file_path2,sep='|',index_col=False)
# only already labeled
df_de = df_de[df_de['cc5'].isna()==False]
print(len(df_de))
# austrian data
df_at = pd.read_csv(path+data_path+file_path3,sep='|',index_col=False)
# only already labeled
df_at = df_at[df_at['cc5'].isna()==False]
df_de = df_de.append(df_at)
print(len(df_de))

df_it = pd.read_csv(path+data_path+file_path4,sep='|',index_col=False)
print(len(df_it))
df_fr = df_fr.sample(frac=1).reset_index(drop=True)
df_de = df_de.sample(frac=1).reset_index(drop=True)

22833
21903
31463
1000


### Exclude non-food?

In [170]:
no_Classes = 75
exclude_non_food = False
if exclude_non_food:
    df_fr = df_fr[df_fr['cc5']!='9999_Non-Food']
    df_de = df_de[df_de['cc5']!='9999_Non-Food']
    df_at = df_it[df_it['cc5']!='9999_Non-Food']
    no_Classes = 74
    print(len(df_fr))
    print(len(df_de))
    print(len(df_at))

decide which columns should be used for training

In [171]:
var = 'name'
#df_fr['text'] = df_fr[var].fillna('unknown') 
#df_de['text'] = df_de[var].fillna('unknown') 
#df_at['text'] = df_at[var].fillna('unknown') 

var1 = 'categ'
var2 = 'words_from_url'
#df_fr['text'] = df_fr[var1].fillna('unknown')  + ' <sep> ' + df_fr[var2].fillna('unknown') 
#df_de['text'] = df_de[var1].fillna('unknown')  + ' <sep> ' + df_de[var2].fillna('unknown') 
#df_at['text'] = df_at[var1].fillna('unknown')  + ' <sep> ' + df_at[var2].fillna('unknown') 


var1 = 'categ'
var2 = 'words_from_url' 
var3 = 'name'
#var1 = 'name' 
#var2 = 'categ' 
#var3 = 'words_from_url' 
df_fr['text'] = df_fr[var1].fillna('unknown')  + ' <sep> ' + df_fr[var2].fillna('unknown')  + ' <sep> ' + df_fr[var3].fillna('unknown') 
df_de['text'] = df_de[var1].fillna('unknown')  + ' <sep> ' + df_de[var2].fillna('unknown')  + ' <sep> ' + df_de[var3].fillna('unknown') 
df_at['text'] = df_at[var1].fillna('unknown')  + ' <sep> ' + df_at[var2].fillna('unknown')  + ' <sep> ' + df_at[var3].fillna('unknown') 
df_it['text'] = df_it[var1].fillna('unknown')  + ' <sep> ' + df_it[var2].fillna('unknown')  + ' <sep> ' + df_it[var3].fillna('unknown') 

#df_fr['text'] = df_fr[var1].fillna('unknown')  + df_fr[var2].fillna('unknown')  + df_fr[var3].fillna('unknown') 
#df_de['text'] = df_de[var1].fillna('unknown')  + df_de[var2].fillna('unknown')  + df_de[var3].fillna('unknown') 
#df_at['text'] = df_at[var1].fillna('unknown')  + df_at[var2].fillna('unknown')  + df_at[var3].fillna('unknown') 

#df_fr['text'] = ' <fr> ' + df_fr['name'] + ' <sep> ' + df_fr['categ'].fillna('unknown') + ' <sep> ' + df_fr['words_from_url'].fillna('unknown') + ' <sep> ' + df_fr['prod_desc'].fillna('unknown') 
#df_de['text'] = ' <de> ' + df_de['name'] + ' <sep> ' + df_de['categ'].fillna('unknown') + ' <sep> ' + df_de['words_from_url'].fillna('unknown') + ' <sep> ' + df_de['prod_desc'].fillna('unknown') 
#df_at['text'] = ' <de> ' + df_at['name'] + ' <sep> ' + df_at['categ'].fillna('unknown') + ' <sep> ' + df_at['words_from_url'].fillna('unknown') + ' <sep> ' + df_at['prod_desc'].fillna('unknown') 


In [172]:
import re
rep_dict = {'.':' ',
                        ',': ' ',
                        '&': ' ',
                        '-': ' ',
                        '/': ' ',
                        'ü': 'ue',
                        'ä': 'ae',
                        'ö': 'oe',
                        'ß': 'ss',
                        'ê': 'e',
                        'é': 'e',
                        'è': 'e',
                        'â': 'a',
                        'á': 'a',
                        'à': 'a',
                        'ô':'o',
                        'œ': 'ae',
                        '%': ' percent ',
                        '1': ' one ',
                        '2': ' two ',
                        '3': ' three ',
                        '4': ' four ',
                        '5': ' five ',
                        '6': ' six ',
                        '7': ' seven ',
                        '8': ' eigth ',
                        '9': ' nine ',
                        '0': ' zero ',
                        ' l ':' liter ',
                        ' ml ':' liter ',
                        'api':' ', 'offers':' ', 'images':' ', 'default':' ', 'msmall' :' ','jpg':' '
                        }

def prepro(line):
    if isinstance(line,str):
        text_str = ' '.join(str(t) for t in line.split())
        text_str = text_str.lower()
        for a,b in rep_dict.items():
            text_str = text_str.replace(a,b)
        text_str = re.sub('[^a-zäöüàáâéèêßœ<>]+', ' ', text_str)
    else: 
        text_str = str(line)
    return text_str

In [173]:
df_fr['text'] = df_fr['text'].apply(lambda x:prepro(x))
df_de['text'] = df_de['text'].apply(lambda x:prepro(x))
df_it['text'] = df_it['text'].apply(lambda x:prepro(x))

In [174]:
print('50% quantile no. of words per row french',np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.50))
print('50% quantile no. of words per row german',np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.50))
print('50% quantile no. of words per row italian',np.quantile(df_it['text'].apply(lambda x: len(str(x).split())),.50))

print('90% quantile no. of words per row french',np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.95))
print('90% quantile no. of words per row german',np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.95))
print('90% quantile no. of words per row italian',np.quantile(df_it['text'].apply(lambda x: len(str(x).split())),.95))

seq_len = int(max(np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.90),np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.95)))
print('seq_len',seq_len)

50% quantile no. of words per row french 19.0
50% quantile no. of words per row german 19.0
50% quantile no. of words per row italian 11.0
90% quantile no. of words per row french 34.0
90% quantile no. of words per row german 27.0
90% quantile no. of words per row italian 18.0
seq_len 31


### Split Train Test

In [175]:
from sklearn.model_selection import train_test_split
def split_train_abs(df):
    X_train, X_val_test, y_train, y_val_test  = train_test_split(df['text'], df['cc5'], random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test , y_val_test, train_size=.5,random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train_de, X_val_de, X_test_de, y_train_de, y_val_de, y_test_de = split_train_abs(df_de)
X_train_fr, X_val_fr, X_test_fr, y_train_fr, y_val_fr, y_test_fr = split_train_abs(df_fr)

df_it_text = df_it['text'][df_it['cc5'].isna()==True]
df_it = df_it[df_it['cc5'].isna()==False]

X_it = df_it['text']
y_it = df_it['cc5']
print('de',X_train_de.shape, X_val_de.shape, X_test_de.shape, y_train_de.shape, y_val_de.shape, y_test_de.shape )
print('fr',X_train_fr.shape, X_val_fr.shape, X_test_fr.shape, y_train_fr.shape, y_val_fr.shape, y_test_fr.shape )
print('it',X_it.shape, y_it.shape)

de (23597,) (3933,) (3933,) (23597,) (3933,) (3933,)
fr (17124,) (2854,) (2855,) (17124,) (2854,) (2855,)
it (415,) (415,)


### Embed

In [176]:
import pickle
french = True
german = True
italian = True

embedding_dim = 300

if french:
    fr_git_embed = pickle.load( open(path + '/embeddings/fr_slim_embed_ext.p', "rb" ) ) #fr_slim_embed_ext #fr_muse_align #fr_muse
    #fr_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Thesis_ecb_ecoicop/embeddings/wiki.fr.vec')
if german:
    de_git_embed = pickle.load( open(path + '/embeddings/de_slim_embed_ext.p', "rb" ) ) #de_muse
    #de_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Thesis_ecb_ecoicop/embeddings/wiki.de.vec')
if italian:
    it_git_embed = pickle.load( open(path + '/embeddings/it_slim_embed.p', "rb" ) ) #de_muse
    #de_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Thesis_ecb_ecoicop/embeddings/wiki.de.vec')

v = np.zeros(300)
v[0]=0
fr_git_embed['<sep>'] = v
de_git_embed['<sep>'] = v
it_git_embed['<sep>'] = v

print('de_git_embed',len(de_git_embed.keys()))
print('fr_git_embed',len(fr_git_embed.keys()))
print('it_git_embed',len(it_git_embed.keys()))



de_git_embed 64412
fr_git_embed 43013
it_git_embed 1984


In [177]:
for word in de_git_embed.keys():
    if word in fr_git_embed.keys():
        if word in it_git_embed.keys():
            avg_vec = (de_git_embed[word] + fr_git_embed[word] + it_git_embed[word]) /3
            de_git_embed[word] = avg_vec
            fr_git_embed[word] = avg_vec
            it_git_embed[word] = avg_vec
        else:
            avg_vec = (de_git_embed[word] + fr_git_embed[word] ) /2
            de_git_embed[word] = avg_vec
            fr_git_embed[word] = avg_vec
    elif word in it_git_embed.keys():
        avg_vec = (de_git_embed[word] + it_git_embed[word] ) /2
        de_git_embed[word] = avg_vec
        it_git_embed[word] = avg_vec

for word in fr_git_embed.keys():
    if word in it_git_embed.keys():
        avg_vec = (it_git_embed[word] + fr_git_embed[word] ) /2
        fr_git_embed[word] = avg_vec
        it_git_embed[word] = avg_vec

In [178]:
if False:
    X_train_emb_de = np.array(list(model_helper.text_to_embed(X_train_de, de_git_embed,  seq_len=seq_len)))
    X_val_emb_de = np.array(list(model_helper.text_to_embed(X_val_de,  de_git_embed,  seq_len=seq_len)))
    X_test_emb_de = np.array(list(model_helper.text_to_embed(X_test_de, de_git_embed,  seq_len=seq_len)))

    X_train_emb_fr = np.array(list(model_helper.text_to_embed(X_train_fr, fr_git_embed, seq_len=seq_len)))
    X_val_emb_fr = np.array(list(model_helper.text_to_embed(X_val_fr, fr_git_embed, seq_len=seq_len)))
    X_test_emb_fr = np.array(list(model_helper.text_to_embed(X_test_fr, fr_git_embed, seq_len=seq_len)))

    X_emb_it = np.array(list(model_helper.text_to_embed(X_it, it_git_embed,  seq_len=seq_len)))

    print('de',X_train_emb_de.shape, X_val_emb_de.shape, X_test_emb_de.shape)
    print('fr',X_train_emb_fr.shape, X_val_emb_fr.shape, X_test_emb_fr.shape)
    print('it',X_emb_it.shape)

### Padding

In [179]:
from tensorflow.keras.preprocessing.text import Tokenizer

#tokenizer_de = Tokenizer()
#tokenizer_de.fit_on_texts(X_train_de.append(X_val_de))
#vocab_size_de = len(tokenizer_de.word_index) + 1

#tokenizer_fr = Tokenizer()
#tokenizer_fr.fit_on_texts(X_train_fr.append(X_val_fr))
#vocab_size_fr = len(tokenizer_fr.word_index) + 1

#tokenizer_it = Tokenizer()
#tokenizer_it.fit_on_texts(X_it)
#vocab_size_it = len(tokenizer_it.word_index) + 1

tokenizer_all = Tokenizer()
tokenizer_all.fit_on_texts(X_train_de.append(X_val_de).append(X_train_fr).append(X_val_fr).append(df_it_text))

vocab_size_all = len(tokenizer_all.word_index) + 1
print(vocab_size_de,vocab_size_fr,vocab_size_it,vocab_size_all)

16861 6972 1246 22822


In [180]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
#german
X_train_tokens_de = tokenizer_all.texts_to_sequences(X_train_de)
X_val_tokens_de = tokenizer_all.texts_to_sequences(X_val_de)
X_test_tokens_de = tokenizer_all.texts_to_sequences(X_test_de)

X_train_pad_de = pad_sequences(X_train_tokens_de,maxlen=seq_len, padding='post')
X_val_pad_de = pad_sequences(X_val_tokens_de,maxlen=seq_len, padding='post')
X_test_pad_de = pad_sequences(X_test_tokens_de,maxlen=seq_len, padding='post')

#french
X_train_tokens_fr = tokenizer_all.texts_to_sequences(X_train_fr)
X_val_tokens_fr = tokenizer_all.texts_to_sequences(X_val_fr)
X_test_tokens_fr = tokenizer_all.texts_to_sequences(X_test_fr)

X_train_pad_fr = pad_sequences(X_train_tokens_fr,maxlen=seq_len, padding='post')
X_val_pad_fr = pad_sequences(X_val_tokens_fr,maxlen=seq_len, padding='post')
X_test_pad_fr = pad_sequences(X_test_tokens_fr,maxlen=seq_len, padding='post')

X_tokens_it = tokenizer_all.texts_to_sequences(X_it)
X_pad_it = pad_sequences(X_tokens_it,maxlen=seq_len, padding='post')

In [181]:
unknown = np.random.rand(embedding_dim)

def get_embed_matrix(embed,tokenizer,vocab_size,embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word,i in tokenizer.word_index.items():
        try:
            embedding_matrix[i] = embed[word]
        except KeyError:
            #next
            embedding_matrix[i] = unknown #np.random.rand(embedding_dim)
    return embedding_matrix

embedding_matrix_de = get_embed_matrix(de_git_embed,tokenizer_de,vocab_size_de,embedding_dim)
embedding_matrix_fr = get_embed_matrix(fr_git_embed,tokenizer_fr,vocab_size_fr,embedding_dim)
embedding_matrix_it = get_embed_matrix(it_git_embed,tokenizer_it,vocab_size_it,embedding_dim)

combine_embed = de_git_embed
combine_embed.update(fr_git_embed)
combine_embed.update(it_git_embed)
embedding_matrix_all = get_embed_matrix(combine_embed,tokenizer_all ,vocab_size_all ,embedding_dim)

### LAbel Encoding

In [182]:

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(df_de['cc5'])

def encode_label(y_):
    y__ = encoder.transform(y_)
    y_enc =tf.keras.utils.to_categorical(y__, num_classes=no_Classes, dtype="float32")
    return y_enc

y_train_enc_de = encode_label(y_train_de)
y_val_enc_de = encode_label(y_val_de)
y_test_enc_de = encode_label(y_test_de)

y_train_enc_fr = encode_label(y_train_fr)
y_val_enc_fr = encode_label(y_val_fr)
y_test_enc_fr = encode_label(y_test_fr)

y_enc_it = encode_label(y_it)

print(y_train_enc_de.shape,y_val_enc_de.shape,y_test_enc_de.shape)
print(y_train_enc_fr.shape,y_val_enc_fr.shape,y_test_enc_fr.shape)

(23597, 75) (3933, 75) (3933, 75)
(17124, 75) (2854, 75) (2855, 75)


In [183]:
def get_weigth_dict(y_train):
    weights_dict = dict(zip(y_train.value_counts().index.tolist(),list(len(y_train) / ( len(y_train.unique())  * y_train.value_counts()))))

    class_weight_dict = {}
    for n,lab in enumerate(encoder.classes_):
        try:
            class_weight_dict[n] = weights_dict[lab]
        except:
            class_weight_dict[n] = 1
    return class_weight_dict

class_weight_dict_de = get_weigth_dict(y_train_de)
class_weight_dict_fr = get_weigth_dict(y_train_fr)
class_weight_dict_de_fr = get_weigth_dict(y_train_de.append(y_train_fr))

# Single Language Classifier German

In [184]:
df_results = pd.DataFrame(columns=['model','task','metric','0','100','250','500','1000','2000','5000','10000','15000','25000'])
df_results

,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000


In [201]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [28]:
def tf_idf_log_reg(X_train,y_train,X_test,y_test,no):
  
    idx = np.random.randint(len(X_train), size=no)

    X_train = [str(x) for x in X_train.iloc[idx]] #.replace('<','').replace('>','')
    y_train = y_train.iloc[idx]
    X_test = [str(x) for x in X_test]
    vectorizer  = TfidfVectorizer()
    vectorizer.fit(X_train)
    X_train_vec = vectorizer.transform(X_train)
    X_test_vec  = vectorizer.transform(X_test)

    logreg = LogisticRegression(C=1,max_iter=500, solver='newton-cg')
    logreg.fit(X_train_vec, y_train)

    y_pred_train = logreg.predict(X_train_vec)
    y_pred_test = logreg.predict(X_test_vec)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)
    


##LogReg

In [29]:
task = 'slc_de'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
    tf_idf_log_reg(X_train_de,y_train_de,X_test_de,y_test_de,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]



  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [00:00<00:01,  6.48it/s]

 22%|██▏       | 2/9 [00:00<00:01,  4.74it/s]

 33%|███▎      | 3/9 [00:01<00:01,  3.14it/s]

 44%|████▍     | 4/9 [00:02<00:02,  1.87it/s]

 56%|█████▌    | 5/9 [00:04<00:04,  1.06s/it]

 67%|██████▋   | 6/9 [00:10<00:07,  2.52s/it]

 78%|███████▊  | 7/9 [00:23<00:11,  5.65s/it]

 89%|████████▉ | 8/9 [00:44<00:10, 10.21s/it]

100%|██████████| 9/9 [01:34<00:00, 10.45s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,LogReg,slc_de,accuracy,NaN,0.3379,0.3786,0.478,0.6621,0.7895,0.8963,0.9303,0.9466,0.9621
1,LogReg,slc_de,avg_recall,NaN,0.7296,0.7374,0.8665,0.9056,0.8839,0.9164,0.9248,0.9356,0.9326


## CNN

In [202]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling1D,AvgPool1D,Dropout,AveragePooling1D, Dense,Embedding, Flatten,Reshape, GlobalAveragePooling1D,MaxPool2D, Concatenate,\
Conv1D,Conv2D,BatchNormalization,Add,Masking,GlobalMaxPooling1D,LayerNormalization,Bidirectional,LSTM,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.regularizers import l1,l2
import pickle
import numpy as np

In [187]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,balanced_accuracy_score

def pred_encode(model,X_test):
    y_pred_test = model.predict(X_test)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]
    return y_pred_test

def all_tests(model,de,fr,it):
    y_pred_test = pred_encode(model,de)
    print()
    print('de',round(accuracy_score(y_pred_test, y_test_de),4))
    y_pred_test = pred_encode(model,fr)
    print('fr',round(accuracy_score(y_pred_test, y_test_fr),4))
    y_pred_test = pred_encode(model,it)
    print('it',round(accuracy_score(y_pred_test, y_it),4))

In [102]:
def run_CNN(X_train_emb,y_train_enc,X_val_emb,y_val_enc,X_test_emb,y_test,class_weight_dict,no):
    
    dropout_rate= 0.4
    filter_sizes =  [1,2,5]
    num_filters = 75
    lr = .001
    opt = Adam(lr=lr, decay=lr/150)

    idx = np.random.randint(len(X_train_emb), size=no)
                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = True, mask_zero=False)(input_layer)
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
        maxpool_pool.append(MaxPooling1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)  
    falt_layer = Flatten(name='flat')(pool_layer)
    #dense_layer = Dense(150,activation='relu')(falt_layer)

    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
    de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    print(X_train_emb[idx,:].shape, y_train_enc[idx,:].shape)
    print(de_cnn1d.summary())
    hist = de_cnn1d.fit(x = X_train_emb[idx,:], y = y_train_enc[idx,:],\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 150, batch_size = 64, shuffle = True, class_weight = class_weight_dict,  \
                    callbacks = [early_stopping])
    #verbose =0,

    y_pred_test = pred_encode(de_cnn1d,X_test_emb)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)

    all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)
    

In [103]:
task = 'slc_de'
model = 'CNN1D_max'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
     #run_CNN(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,class_weight_dict_de,obs)
     run_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,class_weight_dict_de,obs)
df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/9 [00:00<?, ?it/s]

(100, 31) (100, 75)
Model: "model_68"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_65 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_158 (Conv1D)             (None, 31, 75)       22575       embedding_65[0][0]               
__________________________________________________________________________________________________
conv1d_159 (Conv1D)             (None, 30, 75)       45075       embedding_65[0][0]               
_______________________________________________________________________







 11%|█         | 1/9 [00:03<00:26,  3.35s/it]

fr 0.0056
fr 0.0032
it 0.0
(250, 31) (250, 75)
Model: "model_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_66 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_161 (Conv1D)             (None, 31, 75)       22575       embedding_66[0][0]               
__________________________________________________________________________________________________
conv1d_162 (Conv1D)             (None, 30, 75)       45075       embedding_66[0][0]               
____________________________________________







 22%|██▏       | 2/9 [00:32<01:17, 11.09s/it]

...







 89%|████████▉ | 8/9 [06:45<01:08, 68.36s/it]

fr 0.9697
fr 0.338
it 0.2843
(25000, 31) (25000, 75)
Model: "model_76"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_73 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_182 (Conv1D)             (None, 31, 75)       22575       embedding_73[0][0]               
__________________________________________________________________________________________________
conv1d_183 (Conv1D)             (None, 30, 75)       45075       embedding_73[0][0]               
______________________________________







100%|██████████| 9/9 [08:54<00:00, 59.36s/it]

fr 0.973
fr 0.2704
it 0.2506


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,CNN1D_max,slc_de,accuracy,NaN,0.0056,0.7051,0.7994,0.8922,0.9283,0.9527,0.9624,0.9697,0.973
1,CNN1D_max,slc_de,avg_recall,NaN,0.0119,0.5919,0.6787,0.8116,0.8348,0.9052,0.9014,0.9432,0.9345


In [105]:
def run_CNN(X_train_emb,y_train_enc,X_val_emb,y_val_enc,X_test_emb,y_test,class_weight_dict,no):
    
    dropout_rate= 0.3
    filter_sizes =  [1,2,5]
    num_filters = 80
    lr = .005
    opt = Adam(lr=lr, decay=lr/150)

    idx = np.random.randint(len(X_train_emb), size=no)
                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
        maxpool_pool.append(MaxPooling1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)  
    falt_layer = Flatten(name='flat')(pool_layer)
    #dense_layer = Dense(150,activation='relu')(falt_layer)

    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
    de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    print(X_train_emb[idx,:].shape, y_train_enc[idx,:].shape)
    print(de_cnn1d.summary())
    hist = de_cnn1d.fit(x = X_train_emb[idx,:], y = y_train_enc[idx,:],\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 150, batch_size = 64, shuffle = True, class_weight = class_weight_dict,  \
                    callbacks = [early_stopping])
    #verbose =0,

    y_pred_test = pred_encode(de_cnn1d,X_test_emb)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)

    all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)
    

In [106]:
task = 'slc_de'
model = 'CNN1D_WOET'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
     #run_CNN(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,class_weight_dict_de,obs)
     run_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,class_weight_dict_de,obs)
df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/9 [00:00<?, ?it/s]

(100, 31) (100, 75)
Model: "model_77"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_74 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_185 (Conv1D)             (None, 31, 80)       24080       embedding_74[0][0]               
__________________________________________________________________________________________________
conv1d_186 (Conv1D)             (None, 30, 80)       48080       embedding_74[0][0]               
_______________________________________________________________________







 11%|█         | 1/9 [00:02<00:23,  2.99s/it]

it 0.0
(250, 31) (250, 75)
Model: "model_78"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_75 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_188 (Conv1D)             (None, 31, 80)       24080       embedding_75[0][0]               
__________________________________________________________________________________________________
conv1d_189 (Conv1D)             (None, 30, 80)       48080       embedding_75[0][0]               
________________________________________________________________







 22%|██▏       | 2/9 [00:14<00:37,  5.42s/it]

...







 89%|████████▉ | 8/9 [01:42<00:16, 16.15s/it]

it 0.2096
(25000, 31) (25000, 75)
Model: "model_85"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_82 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_209 (Conv1D)             (None, 31, 80)       24080       embedding_82[0][0]               
__________________________________________________________________________________________________
conv1d_210 (Conv1D)             (None, 30, 80)       48080       embedding_82[0][0]               
_________________________________________________________







100%|██████████| 9/9 [02:08<00:00, 14.30s/it]


de 0.9677
fr 0.248
it 0.188


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
2,CNN1D_WOET,slc_de,accuracy,NaN,0.0173,0.7061,0.7849,0.8678,0.9204,0.9466,0.9603,0.9629,0.9677
3,CNN1D_WOET,slc_de,avg_recall,NaN,0.2246,0.6136,0.654,0.7801,0.8632,0.8962,0.9177,0.9294,0.9223


In [107]:
 def run_CNN(X_train_emb,y_train_enc,X_val_emb,y_val_enc,X_test_emb,y_test,class_weight_dict,no):
    
    dropout_rate= 0.2
    filter_sizes =  [1,2,5]
    num_filters = 80
    lr = .01
    opt = Adam(lr=lr, decay=lr/150)

    idx = np.random.randint(len(X_train_emb), size=no)
                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
        maxpool_pool.append(AvgPool1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)  
    falt_layer = Flatten(name='flat')(pool_layer)
    #dense_layer = Dense(150,activation='relu')(falt_layer)

    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
    de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    print(X_train_emb[idx,:].shape, y_train_enc[idx,:].shape)
    print(de_cnn1d.summary())
    hist = de_cnn1d.fit(x = X_train_emb[idx,:], y = y_train_enc[idx,:],\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 150, batch_size = 64, shuffle = True, class_weight = class_weight_dict,  \
                    callbacks = [early_stopping])
    #verbose =0,

    y_pred_test = pred_encode(de_cnn1d,X_test_emb)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)

    all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)
    

In [108]:
task = 'slc_de'
model = 'CNN1D_WOET_Avg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
     #run_CNN(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,class_weight_dict_de,obs)
     run_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,class_weight_dict_de,obs)
df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/9 [00:00<?, ?it/s]

(100, 31) (100, 75)
Model: "model_86"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_83 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_212 (Conv1D)             (None, 31, 80)       24080       embedding_83[0][0]               
__________________________________________________________________________________________________
conv1d_213 (Conv1D)             (None, 30, 80)       48080       embedding_83[0][0]               
_______________________________________________________________________







 11%|█         | 1/9 [00:02<00:23,  2.98s/it]


de 0.0488
fr 0.0389
it 0.0
(250, 31) (250, 75)
Model: "model_87"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_84 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_215 (Conv1D)             (None, 31, 80)       24080       embedding_84[0][0]               
__________________________________________________________________________________________________
conv1d_216 (Conv1D)             (None, 30, 80)       48080       embedding_84[0][0]               
___________________________________________







 22%|██▏       | 2/9 [00:15<00:40,  5.84s/it]

...







 89%|████████▉ | 8/9 [02:01<00:20, 20.47s/it]


de 0.9575
fr 0.4497
it 0.2024
(25000, 31) (25000, 75)
Model: "model_94"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_91 (Embedding)        (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_236 (Conv1D)             (None, 31, 80)       24080       embedding_91[0][0]               
__________________________________________________________________________________________________
conv1d_237 (Conv1D)             (None, 30, 80)       48080       embedding_91[0][0]               
____________________________________







100%|██████████| 9/9 [02:39<00:00, 17.77s/it]


de 0.9626
fr 0.373
it 0.1759


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
4,CNN1D_WOET_Avg,slc_de,accuracy,NaN,0.0488,0.6031,0.6929,0.8029,0.8973,0.9265,0.9522,0.9575,0.9626
5,CNN1D_WOET_Avg,slc_de,avg_recall,NaN,0.3273,0.4711,0.5453,0.6862,0.8111,0.8668,0.9071,0.9108,0.9136


## RNN

In [109]:
def run_RNN(X_train_emb,y_train_enc,X_val_emb,y_val_enc,X_test_emb,y_test,class_weight_dict,no):
    
    dropout_rate= .3
    lr = .005
    opt = Adam(lr=lr, decay=lr/150)

    idx = np.random.randint(len(X_train_emb), size=no)
                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)
    lstm_layer = Bidirectional(LSTM(128,activation = 'tanh',recurrent_activation = 'sigmoid', dropout = dropout_rate,recurrent_dropout = 0, unroll = False,use_bias = True))(embedd_seq)
    drop_layer = Dropout(dropout_rate)(lstm_layer)

    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_rnn = Model(inputs = [input_layer], outputs = pred_layer)
    de_rnn.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_rnn.fit(x = X_train_emb[idx,:], y = y_train_enc[idx,:],\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 100, batch_size = 64, shuffle = True, class_weight = class_weight_dict,  \
                    callbacks = [early_stopping])
    #verbose =0,
    y_pred_test = pred_encode(de_rnn,X_test_emb)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)

    all_tests(de_rnn,X_test_pad_de,X_test_pad_fr,X_pad_it)

In [110]:
task = 'slc_de'
model = 'RNN'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
     #run_RNN(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,class_weight_dict_de,obs)
     run_RNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,class_weight_dict_de,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/9 [00:00<?, ?it/s]

Epoch 1/100
2/2 [==============================] - 1s 419ms/step - loss: 3.6910 - accuracy: 0.0200 - val_loss: 4.2713 - val_accuracy: 0.0071
Epoch 2/100
2/2 [==============================] - 0s 133ms/step - loss: 3.4595 - accuracy: 0.0300 - val_loss: 4.1538 - val_accuracy: 0.0066
Epoch 3/100
2/2 [==============================] - 0s 137ms/step - loss: 3.1636 - accuracy: 0.0600 - val_loss: 4.3496 - val_accuracy: 0.0074
Epoch 4/100
2/2 [==============================] - 0s 134ms/step - loss: 2.8513 - accuracy: 0.0400 - val_loss: 4.5494 - val_accuracy: 0.0081
Epoch 5/100
2/2 [==============================] - 0s 134ms/step - loss: 2.7437 - accuracy: 0.0400 - val_loss: 4.5832 - val_accuracy: 0.0081
Epoch 6/100
2/2 [==============================] - 0s 135ms/step - loss: 2.5973 - accuracy: 0.0600 - val_loss: 4.5386 - val_accuracy: 0.0150
Epoch 7/100
2/2 [==============================] - 0s 141ms/step - loss: 2.5038 - accuracy: 0.0900 - val_loss: 4.5216 - val_accuracy: 0.0280

de 0.0313








 11%|█         | 1/9 [00:07<00:59,  7.48s/it]

fr 0.0046
it 0.0
Epoch 1/100
4/4 [==============================] - 1s 207ms/step - loss: 4.9486 - accuracy: 0.0080 - val_loss: 4.3384 - val_accuracy: 0.0173
Epoch 2/100
4/4 [==============================] - 0s 71ms/step - loss: 4.5456 - accuracy: 0.0280 - val_loss: 4.3967 - val_accuracy: 0.0076
Epoch 3/100
4/4 [==============================] - 0s 74ms/step - loss: 4.2557 - accuracy: 0.0320 - val_loss: 4.5460 - val_accuracy: 0.0076
Epoch 4/100
4/4 [==============================] - 0s 72ms/step - loss: 4.0454 - accuracy: 0.0320 - val_loss: 4.4252 - val_accuracy: 0.0183
Epoch 5/100
4/4 [==============================] - 0s 72ms/step - loss: 3.7006 - accuracy: 0.0640 - val_loss: 4.1830 - val_accuracy: 0.0244
Epoch 6/100
4/4 [==============================] - 0s 72ms/step - loss: 3.4583 - accuracy: 0.0880 - val_loss: 4.2377 - val_accuracy: 0.0361
Epoch 7/100
4/4 [==============================] - 0s 71ms/step - loss: 3.2835 - accuracy: 0.0880 - val_loss: 4.1489 - val_accuracy: 0.0501
Ep







 22%|██▏       | 2/9 [00:22<01:08,  9.84s/it]

...







 89%|████████▉ | 8/9 [02:59<00:28, 28.93s/it]

fr 0.1947
it 0.1976
Epoch 1/100
391/391 [==============================] - 4s 11ms/step - loss: 2.0362 - accuracy: 0.5782 - val_loss: 0.7979 - val_accuracy: 0.8019
Epoch 2/100
391/391 [==============================] - 3s 8ms/step - loss: 0.8348 - accuracy: 0.8394 - val_loss: 0.4255 - val_accuracy: 0.8914
Epoch 3/100
391/391 [==============================] - 3s 8ms/step - loss: 0.5613 - accuracy: 0.8879 - val_loss: 0.3476 - val_accuracy: 0.9166
Epoch 4/100
391/391 [==============================] - 3s 8ms/step - loss: 0.4246 - accuracy: 0.9136 - val_loss: 0.3032 - val_accuracy: 0.9245
Epoch 5/100
391/391 [==============================] - 3s 8ms/step - loss: 0.3770 - accuracy: 0.9248 - val_loss: 0.2704 - val_accuracy: 0.9336
Epoch 6/100
391/391 [==============================] - 3s 8ms/step - loss: 0.3146 - accuracy: 0.9369 - val_loss: 0.2571 - val_accuracy: 0.9385
Epoch 7/100
391/391 [==============================] - 3s 8ms/step - loss: 0.2525 - accuracy: 0.9461 - val_loss: 0.2418 -







100%|██████████| 9/9 [04:13<00:00, 28.12s/it]

fr 0.2028
it 0.1711


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
6,RNN,slc_de,accuracy,NaN,0.0313,0.5578,0.6995,0.8408,0.8935,0.926,0.9433,0.9408,0.9621
7,RNN,slc_de,avg_recall,NaN,0.1149,0.3815,0.5266,0.6932,0.7806,0.8491,0.8711,0.8845,0.9226


In [111]:
task = 'slc_de'
df_results[(df_results['task']==task)]

,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,CNN1D_max,slc_de,accuracy,NaN,0.0056,0.7051,0.7994,0.8922,0.9283,0.9527,0.9624,0.9697,0.973
1,CNN1D_max,slc_de,avg_recall,NaN,0.0119,0.5919,0.6787,0.8116,0.8348,0.9052,0.9014,0.9432,0.9345
2,CNN1D_WOET,slc_de,accuracy,NaN,0.0173,0.7061,0.7849,0.8678,0.9204,0.9466,0.9603,0.9629,0.9677
3,CNN1D_WOET,slc_de,avg_recall,NaN,0.2246,0.6136,0.654,0.7801,0.8632,0.8962,0.9177,0.9294,0.9223
4,CNN1D_WOET_Avg,slc_de,accuracy,NaN,0.0488,0.6031,0.6929,0.8029,0.8973,0.9265,0.9522,0.9575,0.9626
5,CNN1D_WOET_Avg,slc_de,avg_recall,NaN,0.3273,0.4711,0.5453,0.6862,0.8111,0.8668,0.9071,0.9108,0.9136
6,RNN,slc_de,accuracy,NaN,0.0313,0.5578,0.6995,0.8408,0.8935,0.926,0.9433,0.9408,0.9621
7,RNN,slc_de,avg_recall,NaN,0.1149,0.3815,0.5266,0.6932,0.7806,0.8491,0.8711,0.8845,0.9226


# Single Language Classifier French

##LogReg

In [112]:
task = 'slc_fr'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)
    
for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
    tf_idf_log_reg(X_train_fr,y_train_fr,X_test_fr,y_test_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/8 [00:00<?, ?it/s]





 12%|█▎        | 1/8 [00:00<00:00,  9.08it/s]





 25%|██▌       | 2/8 [00:00<00:00,  6.01it/s]





 38%|███▊      | 3/8 [00:00<00:01,  3.56it/s]





 50%|█████     | 4/8 [00:01<00:01,  2.07it/s]





 62%|██████▎   | 5/8 [00:03<00:02,  1.14it/s]





 75%|███████▌  | 6/8 [00:08<00:04,  2.01s/it]





 88%|████████▊ | 7/8 [00:18<00:04,  4.51s/it]





100%|██████████| 8/8 [00:34<00:00,  4.37s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
8,LogReg,slc_fr,accuracy,NaN,0.5419,0.5818,0.7114,0.7765,0.8392,0.8942,0.9264,0.9436,NaN
9,LogReg,slc_fr,avg_recall,NaN,0.7418,0.8571,0.9028,0.8399,0.852,0.8573,0.9192,0.9185,NaN


##CNN

In [113]:
task = 'slc_fr'
model = 'CNN1D'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
     run_CNN(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,class_weight_dict_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/8 [00:00<?, ?it/s]

(100, 31) (100, 75)
Model: "model_104"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_101 (Embedding)       (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_239 (Conv1D)             (None, 31, 80)       24080       embedding_101[0][0]              
__________________________________________________________________________________________________
conv1d_240 (Conv1D)             (None, 30, 80)       48080       embedding_101[0][0]              
______________________________________________________________________







 12%|█▎        | 1/8 [00:02<00:17,  2.45s/it]


de 0.0051
fr 0.0126
it 0.0024
(250, 31) (250, 75)
Model: "model_105"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_102 (Embedding)       (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_242 (Conv1D)             (None, 31, 80)       24080       embedding_102[0][0]              
__________________________________________________________________________________________________
conv1d_243 (Conv1D)             (None, 30, 80)       48080       embedding_102[0][0]              
_______________________________________







 25%|██▌       | 2/8 [00:04<00:14,  2.46s/it]

...







 88%|████████▊ | 7/8 [01:09<00:12, 12.88s/it]


de 0.4327
fr 0.9457
it 0.3229
(15000, 31) (15000, 75)
Model: "model_111"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_108 (Embedding)       (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_260 (Conv1D)             (None, 31, 80)       24080       embedding_108[0][0]              
__________________________________________________________________________________________________
conv1d_261 (Conv1D)             (None, 30, 80)       48080       embedding_108[0][0]              
___________________________________







100%|██████████| 8/8 [01:43<00:00, 12.96s/it]


de 0.4409
fr 0.9622
it 0.1639


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
10,CNN1D,slc_fr,accuracy,NaN,0.0126,0.0053,0.7678,0.849,0.8774,0.9236,0.9457,0.9622,NaN
11,CNN1D,slc_fr,avg_recall,NaN,0.1606,0.2097,0.5381,0.6733,0.7294,0.8066,0.8721,0.8948,NaN


## RNN

In [114]:
task = 'slc_fr'
model = 'RNN'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
     run_RNN(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,class_weight_dict_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]







  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/100
2/2 [==============================] - 1s 390ms/step - loss: 4.5872 - accuracy: 0.0200 - val_loss: 4.2380 - val_accuracy: 0.0098
Epoch 2/100
2/2 [==============================] - 0s 102ms/step - loss: 4.2091 - accuracy: 0.0300 - val_loss: 4.3760 - val_accuracy: 0.0042
Epoch 3/100
2/2 [==============================] - 0s 103ms/step - loss: 3.7493 - accuracy: 0.0300 - val_loss: 4.5689 - val_accuracy: 0.0042
Epoch 4/100
2/2 [==============================] - 0s 106ms/step - loss: 3.6050 - accuracy: 0.0300 - val_loss: 4.5680 - val_accuracy: 0.0042
Epoch 5/100
2/2 [==============================] - 0s 109ms/step - loss: 3.5759 - accuracy: 0.0200 - val_loss: 4.4992 - val_accuracy: 0.0035
Epoch 6/100
2/2 [==============================] - 0s 104ms/step - loss: 3.4428 - accuracy: 0.0200 - val_loss: 4.3707 - val_accuracy: 0.0046

de 0.0079








 12%|█▎        | 1/8 [00:06<00:48,  6.96s/it]

fr 0.0056
it 0.0
Epoch 1/100
4/4 [==============================] - 1s 193ms/step - loss: 3.5505 - accuracy: 0.0120 - val_loss: 4.0919 - val_accuracy: 0.0039
Epoch 2/100
4/4 [==============================] - 0s 54ms/step - loss: 3.2804 - accuracy: 0.0120 - val_loss: 4.1136 - val_accuracy: 0.0014
Epoch 3/100
4/4 [==============================] - 0s 55ms/step - loss: 3.0346 - accuracy: 0.0200 - val_loss: 3.8616 - val_accuracy: 0.0280
Epoch 4/100
4/4 [==============================] - 0s 54ms/step - loss: 2.8073 - accuracy: 0.1120 - val_loss: 3.8838 - val_accuracy: 0.0137
Epoch 5/100
4/4 [==============================] - 0s 54ms/step - loss: 2.5790 - accuracy: 0.0360 - val_loss: 3.9112 - val_accuracy: 0.0144
Epoch 6/100
4/4 [==============================] - 0s 56ms/step - loss: 2.4447 - accuracy: 0.0640 - val_loss: 3.8723 - val_accuracy: 0.0168
Epoch 7/100
4/4 [==============================] - 0s 55ms/step - loss: 2.2493 - accuracy: 0.0800 - val_loss: 3.7754 - val_accuracy: 0.0238
Ep







 25%|██▌       | 2/8 [00:20<00:53,  8.98s/it]

...







 88%|████████▊ | 7/8 [01:59<00:20, 20.24s/it]

fr 0.9201
it 0.3253
Epoch 1/100
235/235 [==============================] - 2s 11ms/step - loss: 3.0103 - accuracy: 0.3255 - val_loss: 1.3892 - val_accuracy: 0.6423
Epoch 2/100
235/235 [==============================] - 2s 8ms/step - loss: 1.5089 - accuracy: 0.7115 - val_loss: 0.7793 - val_accuracy: 0.7733
Epoch 3/100
235/235 [==============================] - 2s 8ms/step - loss: 1.0553 - accuracy: 0.8026 - val_loss: 0.7069 - val_accuracy: 0.7915
Epoch 4/100
235/235 [==============================] - 2s 8ms/step - loss: 0.7908 - accuracy: 0.8552 - val_loss: 0.4858 - val_accuracy: 0.8662
Epoch 5/100
235/235 [==============================] - 2s 8ms/step - loss: 0.6314 - accuracy: 0.8763 - val_loss: 0.4644 - val_accuracy: 0.8858
Epoch 6/100
235/235 [==============================] - 2s 8ms/step - loss: 0.6478 - accuracy: 0.8751 - val_loss: 0.4232 - val_accuracy: 0.8872
Epoch 7/100
235/235 [==============================] - 2s 9ms/step - loss: 0.4247 - accuracy: 0.9075 - val_loss: 0.3630 -







100%|██████████| 8/8 [03:07<00:00, 23.47s/it]

fr 0.938
it 0.147


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
12,RNN,slc_fr,accuracy,NaN,0.0056,0.6147,0.7201,0.8126,0.848,0.8935,0.9201,0.938,NaN
13,RNN,slc_fr,avg_recall,NaN,0.0698,0.3826,0.4703,0.5976,0.6713,0.7317,0.7932,0.8389,NaN


In [ ]:
task = 'slc_fr'
df_results[(df_results['task']==task)]

In [130]:
X_it.shape

(415,)

In [134]:
task = 'slc_it'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)
    
tf_idf_log_reg(X_it[:365],y_it[:365],X_it[365:],y_it[365:],500)


In [135]:
df_results

,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,CNN1D_max,slc_de,accuracy,NaN,0.0056,0.7051,0.7994,0.8922,0.9283,0.9527,0.9624,0.9697,0.973
1,CNN1D_max,slc_de,avg_recall,NaN,0.0119,0.5919,0.6787,0.8116,0.8348,0.9052,0.9014,0.9432,0.9345
2,CNN1D_WOET,slc_de,accuracy,NaN,0.0173,0.7061,0.7849,0.8678,0.9204,0.9466,0.9603,0.9629,0.9677
3,CNN1D_WOET,slc_de,avg_recall,NaN,0.2246,0.6136,0.654,0.7801,0.8632,0.8962,0.9177,0.9294,0.9223
4,CNN1D_WOET_Avg,slc_de,accuracy,NaN,0.0488,0.6031,0.6929,0.8029,0.8973,0.9265,0.9522,0.9575,0.9626
5,CNN1D_WOET_Avg,slc_de,avg_recall,NaN,0.3273,0.4711,0.5453,0.6862,0.8111,0.8668,0.9071,0.9108,0.9136
6,RNN,slc_de,accuracy,NaN,0.0313,0.5578,0.6995,0.8408,0.8935,0.926,0.9433,0.9408,0.9621
7,RNN,slc_de,avg_recall,NaN,0.1149,0.3815,0.5266,0.6932,0.7806,0.8491,0.8711,0.8845,0.9226
8,LogReg,slc_fr,accuracy,NaN,0.5419,0.5818,0.7114,0.7765,0.8392,0.8942,0.9264,0.9436,NaN
9,LogReg,slc_fr,avg_recall,NaN,0.7418,0.8571,0.9028,0.8399,0.852,0.8573,0.9192,0.9185,NaN


# Multilingual Training

## CNN DE FR 

### LogReg

In [ ]:
def tf_idf_log_reg(X_train_de,y_train_de,X_train_fr,y_train_fr,X_test_fr,y_test_fr,no):
    
    idx = np.random.randint(len(X_train_fr), size=no)

    X_train_fr = [str(x) for x in X_train_fr.iloc[idx]] 
    y_train_fr = y_train_fr.iloc[idx]

    X_train = X_train_de.append(X_train_fr)
    y_train = y_train_de.append(y_train_fr)

    X_test = [str(x) for x in X_test]
    vectorizer  = TfidfVectorizer()
    vectorizer.fit(X_train)
    X_train_vec = vectorizer.transform(X_train)
    X_test_vec  = vectorizer.transform(X_test)

    logreg = LogisticRegression(C=1,max_iter=500, solver='newton-cg')#,class_weight=weights_dict)
    logreg.fit(X_train_vec, y_train)

    y_pred_train = logreg.predict(X_train_vec)
    y_pred_test = logreg.predict(X_test_vec)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)
    


In [ ]:
task = 'mlc_defr'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500]):#,1000,2000,5000,10000,15000]):
    tf_idf_log_reg(X_train_de,y_train_de,X_train_fr,y_train_fr,X_test_fr,y_test_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]

### CNN

In [122]:
 def run_ml_CNN1D(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,
             X_train_emb_fr,y_train_enc_fr,X_val_emb_fr,y_val_enc_fr,X_test_emb_fr,y_test_fr,
             class_weight_dict,no):
   
    idx = np.random.randint(len(X_train_emb_fr), size=no)
    idx_de = np.random.randint(len(X_train_emb_de), size=len(X_train_emb_fr))

    X_train_emb =  np.concatenate((X_train_emb_de[idx_de,:], X_train_emb_fr[idx,:]))
    y_train_enc =  np.concatenate((y_train_enc_de[idx_de,:], y_train_enc_fr[idx,:]))

    imbala = int(len(X_train_emb_fr)/no)+1
    for n in range(1,imbala):
        X_train_emb =  np.concatenate((X_train_emb, X_train_emb_fr[idx,:]))
        y_train_enc =  np.concatenate((y_train_enc, y_train_enc_fr[idx,:]))

    X_val_emb =  np.concatenate((X_val_emb_de, X_val_emb_fr))
    y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))
    print(imbala,len(X_train_emb_de),len(X_train_emb))

    dropout_rate= .2
    filter_sizes = [1,2,5]
    num_filters = 100
    lr = .005
    opt = Adam(lr=lr, decay=lr/150)

                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
        maxpool_pool.append(AveragePooling1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)   
    falt_layer = Flatten(name='flat')(pool_layer)
    #dense_layer = Dense(150,activation='tanh')(falt_layer)

    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn1d = Model(inputs = [input_layer], outputs = [pred_layer])

    de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
    de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_cnn1d.fit(x = X_train_emb, y = y_train_enc,\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 50, batch_size = 64, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    

    y_pred_test = pred_encode(de_cnn1d,X_test_emb_fr)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test_fr),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test_fr),4)
    
    all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)



In [123]:
task = 'mlc_defr'
model = 'CNN1D'
metrics = ['accuracy','avg_recall']

for m in metrics:
  if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
     run_ml_CNN1D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,\
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,\
             class_weight_dict_de_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]









  0%|          | 0/6 [00:00<?, ?it/s]

172 23597 34324
Epoch 1/50
537/537 [==============================] - 3s 5ms/step - loss: 1.9177 - accuracy: 0.6165 - val_loss: 1.6556 - val_accuracy: 0.6524
Epoch 2/50
537/537 [==============================] - 2s 5ms/step - loss: 0.7263 - accuracy: 0.8692 - val_loss: 1.1895 - val_accuracy: 0.7620
Epoch 3/50
537/537 [==============================] - 2s 5ms/step - loss: 0.4454 - accuracy: 0.9132 - val_loss: 1.1384 - val_accuracy: 0.7945
Epoch 4/50
537/537 [==============================] - 2s 5ms/step - loss: 0.3191 - accuracy: 0.9333 - val_loss: 1.1552 - val_accuracy: 0.7950
Epoch 5/50
537/537 [==============================] - 2s 5ms/step - loss: 0.2445 - accuracy: 0.9490 - val_loss: 1.0999 - val_accuracy: 0.8008
Epoch 6/50
537/537 [==============================] - 2s 5ms/step - loss: 0.2011 - accuracy: 0.9587 - val_loss: 1.0465 - val_accuracy: 0.8292
Epoch 7/50
537/537 [==============================] - 2s 5ms/step - loss: 0.1689 - accuracy: 0.9641 - val_loss: 1.0789 - val_accurac









 17%|█▋        | 1/6 [00:36<03:02, 36.53s/it]


de 0.9547
fr 0.711
it 0.1928
69 23597 34374
Epoch 1/50
538/538 [==============================] - 3s 5ms/step - loss: 2.2123 - accuracy: 0.5454 - val_loss: 1.3075 - val_accuracy: 0.6996
Epoch 2/50
538/538 [==============================] - 2s 5ms/step - loss: 0.7418 - accuracy: 0.8580 - val_loss: 0.9994 - val_accuracy: 0.7645
Epoch 3/50
538/538 [==============================] - 2s 5ms/step - loss: 0.4706 - accuracy: 0.9061 - val_loss: 0.8907 - val_accuracy: 0.8124
Epoch 4/50
538/538 [==============================] - 2s 5ms/step - loss: 0.3327 - accuracy: 0.9307 - val_loss: 0.7481 - val_accuracy: 0.8457
Epoch 5/50
538/538 [==============================] - 2s 5ms/step - loss: 0.2603 - accuracy: 0.9450 - val_loss: 0.7435 - val_accuracy: 0.8432
Epoch 6/50
538/538 [==============================] - 2s 5ms/step - loss: 0.2087 - accuracy: 0.9540 - val_loss: 0.7320 - val_accuracy: 0.8588
Epoch 7/50
538/538 [==============================] - 2s 5ms/step - loss: 0.1804 - accuracy: 0.9620 - v









 33%|███▎      | 2/6 [01:08<02:20, 35.02s/it]

...


de 0.9458
fr 0.9037
it 0.2024
4 23597 37124
Epoch 1/50
581/581 [==============================] - 3s 5ms/step - loss: 2.4971 - accuracy: 0.5310 - val_loss: 1.1655 - val_accuracy: 0.7040
Epoch 2/50
581/581 [==============================] - 3s 5ms/step - loss: 1.0503 - accuracy: 0.7976 - val_loss: 0.6720 - val_accuracy: 0.8444
Epoch 3/50
581/581 [==============================] - 3s 5ms/step - loss: 0.6823 - accuracy: 0.8598 - val_loss: 0.5281 - val_accuracy: 0.8774
Epoch 4/50
581/581 [==============================] - 3s 5ms/step - loss: 0.4898 - accuracy: 0.8937 - val_loss: 0.4442 - val_accuracy: 0.8910
Epoch 5/50
581/581 [==============================] - 3s 5ms/step - loss: 0.3706 - accuracy: 0.9156 - val_loss: 0.4099 - val_accuracy: 0.9058
Epoch 6/50
581/581 [==============================] - 3s 5ms/step - loss: 0.3008 - accuracy: 0.9298 - val_loss: 0.3629 - val_accuracy: 0.9196
Epoch 7/50
581/581 [==============================] - 3s 5ms/step - loss: 0.2416 - accuracy: 0.9418 - v









100%|██████████| 6/6 [03:52<00:00, 42.73s/it]


de 0.9509
fr 0.9317
it 0.2434


100%|██████████| 6/6 [03:52<00:00, 38.79s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
14,CNN1D,mlc_defr,accuracy,NaN,0.711,0.7944,0.8231,0.8718,0.9037,0.9317,NaN,NaN,NaN
15,CNN1D,mlc_defr,avg_recall,NaN,0.4971,0.6659,0.6697,0.7077,0.7974,0.8259,NaN,NaN,NaN


### RNN

In [124]:
 def run_ml_RNN(X_train_emb_de,y_train_enc_de,X_val_emb_de,y_val_enc_de,X_test_emb_de,y_test_de,
             X_train_emb_fr,y_train_enc_fr,X_val_emb_fr,y_val_enc_fr,X_test_emb_fr,y_test_fr,
             class_weight_dict,no):
   
    idx = np.random.randint(len(X_train_emb_fr), size=no)
    #idx_de = np.random.randint(len(X_train_emb_de), size=len(X_train_emb_fr))

    X_train_emb =  np.concatenate((X_train_emb_de, X_train_emb_fr[idx,:]))
    y_train_enc =  np.concatenate((y_train_enc_de, y_train_enc_fr[idx,:]))

    imbala = int(len(X_train_emb_fr)/no)+1
    for n in range(1,imbala):
        X_train_emb =  np.concatenate((X_train_emb, X_train_emb_fr[idx,:]))
        y_train_enc =  np.concatenate((y_train_enc, y_train_enc_fr[idx,:]))

    X_val_emb =  np.concatenate((X_val_emb_de, X_val_emb_fr))
    y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))
    print(imbala,len(X_train_emb_de),len(X_train_emb))

    dropout_rate= .5
    lr = .005
    opt = Adam(lr=lr, decay=lr/100)

                          
    input_layer = Input(shape = (seq_len,), name='text_input')
    embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)

    lstm_layer = Bidirectional(LSTM(256,activation = 'tanh',recurrent_activation = 'sigmoid', dropout = dropout_rate,recurrent_dropout = 0, unroll = False,use_bias = True))(embedd_seq)
    drop_layer = Dropout(dropout_rate)(lstm_layer)

    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn1d = Model(inputs = [input_layer], outputs = [pred_layer])

    de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
    de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_cnn1d.fit(x = X_train_emb, y = y_train_enc,\
                    validation_data = (X_val_emb, y_val_enc), \
                    epochs = 50, batch_size = 64, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test = de_cnn1d.predict(X_test_emb_fr)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test_fr),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test_fr),4)

    all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)

In [125]:
task = 'mlc_defr'
model = 'RNN'
metrics = ['accuracy','avg_recall']

for m in metrics:
  if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
     run_ml_RNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,\
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,\
             class_weight_dict_de_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]










  0%|          | 0/6 [00:00<?, ?it/s]

172 23597 40797
Epoch 1/50
638/638 [==============================] - 7s 11ms/step - loss: 1.6411 - accuracy: 0.6172 - val_loss: 1.5423 - val_accuracy: 0.6676
Epoch 2/50
638/638 [==============================] - 7s 11ms/step - loss: 0.7021 - accuracy: 0.8656 - val_loss: 1.5489 - val_accuracy: 0.7336
Epoch 3/50
638/638 [==============================] - 7s 10ms/step - loss: 0.4743 - accuracy: 0.9041 - val_loss: 1.3965 - val_accuracy: 0.7579
Epoch 4/50
638/638 [==============================] - 7s 11ms/step - loss: 0.3761 - accuracy: 0.9240 - val_loss: 1.4500 - val_accuracy: 0.7469
Epoch 5/50
638/638 [==============================] - 7s 11ms/step - loss: 0.3374 - accuracy: 0.9293 - val_loss: 1.4557 - val_accuracy: 0.7693
Epoch 6/50
638/638 [==============================] - 7s 10ms/step - loss: 0.2774 - accuracy: 0.9432 - val_loss: 1.4557 - val_accuracy: 0.7785
Epoch 7/50
638/638 [==============================] - 7s 10ms/step - loss: 0.2668 - accuracy: 0.9471 - val_loss: 1.5287 - val_









 17%|█▋        | 1/6 [00:59<04:55, 59.16s/it]

fr 0.5243
it 0.1494
69 23597 40847
Epoch 1/50
639/639 [==============================] - 7s 11ms/step - loss: 1.8515 - accuracy: 0.5606 - val_loss: 1.6542 - val_accuracy: 0.6154
Epoch 2/50
639/639 [==============================] - 7s 10ms/step - loss: 0.7451 - accuracy: 0.8486 - val_loss: 1.0850 - val_accuracy: 0.7903
Epoch 3/50
639/639 [==============================] - 7s 10ms/step - loss: 0.5114 - accuracy: 0.8977 - val_loss: 0.9493 - val_accuracy: 0.8142
Epoch 4/50
639/639 [==============================] - 7s 10ms/step - loss: 0.4029 - accuracy: 0.9170 - val_loss: 0.9799 - val_accuracy: 0.8257
Epoch 5/50
639/639 [==============================] - 7s 10ms/step - loss: 0.3380 - accuracy: 0.9320 - val_loss: 0.9572 - val_accuracy: 0.8276
Epoch 6/50
639/639 [==============================] - 7s 10ms/step - loss: 0.2993 - accuracy: 0.9405 - val_loss: 0.9716 - val_accuracy: 0.8384
Epoch 7/50
639/639 [==============================] - 7s 11ms/step - loss: 0.2626 - accuracy: 0.9464 - val_









 33%|███▎      | 2/6 [02:32<04:37, 69.32s/it]

...









 83%|████████▎ | 5/6 [06:55<01:20, 80.17s/it]

fr 0.8687
it 0.0795
4 23597 43597
Epoch 1/50
682/682 [==============================] - 8s 11ms/step - loss: 2.2241 - accuracy: 0.4987 - val_loss: 0.9964 - val_accuracy: 0.7416
Epoch 2/50
682/682 [==============================] - 8s 11ms/step - loss: 1.0074 - accuracy: 0.7954 - val_loss: 0.5311 - val_accuracy: 0.8662
Epoch 3/50
682/682 [==============================] - 7s 11ms/step - loss: 0.6847 - accuracy: 0.8463 - val_loss: 0.4796 - val_accuracy: 0.8817
Epoch 4/50
682/682 [==============================] - 7s 11ms/step - loss: 0.5128 - accuracy: 0.8814 - val_loss: 0.4017 - val_accuracy: 0.9023
Epoch 5/50
682/682 [==============================] - 7s 11ms/step - loss: 0.4264 - accuracy: 0.9013 - val_loss: 0.4055 - val_accuracy: 0.9078
Epoch 6/50
682/682 [==============================] - 7s 11ms/step - loss: 0.3640 - accuracy: 0.9117 - val_loss: 0.4072 - val_accuracy: 0.9067
Epoch 7/50
682/682 [==============================] - 7s 11ms/step - loss: 0.3308 - accuracy: 0.9190 - val_l









100%|██████████| 6/6 [09:33<00:00, 95.58s/it] 

fr 0.9191
it 0.1277


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
16,RNN,mlc_defr,accuracy,NaN,0.5243,0.7226,0.7737,0.82,0.8687,0.9191,NaN,NaN,NaN
17,RNN,mlc_defr,avg_recall,NaN,0.3199,0.4196,0.5128,0.6007,0.6987,0.7952,NaN,NaN,NaN


## CNN FR DE

### LogReg


In [ ]:
def tf_idf_log_reg(X_train_de,y_train_de,X_train_fr,y_train_fr,X_test_de,y_test_de,no):
    
    idx = np.random.randint(len(X_train_de), size=no)

    X_train_de = [str(x) for x in X_train_de.iloc[idx]] 
    y_train_de = y_train_de.iloc[idx]
    X_train_fr = [str(x) for x in X_train_fr] 

    X_train = list(X_train_fr).extend(list(X_train_de))
    y_train = list(y_train_fr).extend(list(y_train_de))

    X_test = [str(x) for x in X_test_de]
    vectorizer  = TfidfVectorizer()
    vectorizer.fit(X_train)
    X_train_vec = vectorizer.transform(X_train)
    X_test_vec  = vectorizer.transform(X_test)

    logreg = LogisticRegression(C=1,max_iter=500, solver='newton-cg')#,class_weight=weights_dict)
    logreg.fit(X_train_vec, y_train)

    y_pred_train = logreg.predict(X_train_vec)
    y_pred_test = logreg.predict(X_test_vec)

    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test, y_test),4)
    df_results[str(no)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test, y_test),4)
    


In [ ]:
task = 'mlc_frde'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
    tf_idf_log_reg(X_train_de,y_train_de,X_train_fr,y_train_fr,X_test_de,y_test_de,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]

### CNN


In [127]:
task = 'mlc_frde'
model = 'CNN1D'
metrics = ['accuracy','avg_recall']

for m in metrics:
  if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000,25000]):
     run_ml_CNN1D(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,\
                  X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,\
                  class_weight_dict_de_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]









  0%|          | 0/9 [00:00<?, ?it/s]

236 17124 47197
Epoch 1/50
738/738 [==============================] - 3s 5ms/step - loss: 1.4156 - accuracy: 0.6880 - val_loss: 1.4801 - val_accuracy: 0.6831
Epoch 2/50
738/738 [==============================] - 3s 4ms/step - loss: 0.3606 - accuracy: 0.9254 - val_loss: 1.3547 - val_accuracy: 0.7252
Epoch 3/50
738/738 [==============================] - 3s 5ms/step - loss: 0.2254 - accuracy: 0.9501 - val_loss: 1.4066 - val_accuracy: 0.7234
Epoch 4/50
738/738 [==============================] - 3s 5ms/step - loss: 0.1626 - accuracy: 0.9612 - val_loss: 1.4175 - val_accuracy: 0.7426
Epoch 5/50
738/738 [==============================] - 3s 4ms/step - loss: 0.1266 - accuracy: 0.9679 - val_loss: 1.3779 - val_accuracy: 0.7464
Epoch 6/50
738/738 [==============================] - 3s 5ms/step - loss: 0.1003 - accuracy: 0.9736 - val_loss: 1.3306 - val_accuracy: 0.7669
Epoch 7/50
738/738 [==============================] - 3s 4ms/step - loss: 0.0797 - accuracy: 0.9783 - val_loss: 1.3948 - val_accurac









 11%|█         | 1/9 [00:51<06:52, 51.52s/it]


de 0.6344
fr 0.9601
it 0.1783
95 17124 47347
Epoch 1/50
740/740 [==============================] - 3s 5ms/step - loss: 1.5801 - accuracy: 0.6716 - val_loss: 1.1694 - val_accuracy: 0.7438
Epoch 2/50
740/740 [==============================] - 3s 4ms/step - loss: 0.4092 - accuracy: 0.9179 - val_loss: 1.0066 - val_accuracy: 0.7977
Epoch 3/50
740/740 [==============================] - 3s 4ms/step - loss: 0.2464 - accuracy: 0.9494 - val_loss: 0.9879 - val_accuracy: 0.8062
Epoch 4/50
740/740 [==============================] - 3s 4ms/step - loss: 0.1739 - accuracy: 0.9600 - val_loss: 1.0074 - val_accuracy: 0.8126
Epoch 5/50
740/740 [==============================] - 3s 4ms/step - loss: 0.1311 - accuracy: 0.9679 - val_loss: 0.9299 - val_accuracy: 0.8242
Epoch 6/50
740/740 [==============================] - 3s 5ms/step - loss: 0.1083 - accuracy: 0.9731 - val_loss: 0.9400 - val_accuracy: 0.8337
Epoch 7/50
740/740 [==============================] - 3s 5ms/step - loss: 0.0894 - accuracy: 0.9781 - 









 22%|██▏       | 2/9 [01:27<05:27, 46.78s/it]

...









 89%|████████▉ | 8/9 [07:23<01:02, 62.76s/it]


de 0.9481
fr 0.9531
it 0.2795
1 17124 48597
Epoch 1/50
760/760 [==============================] - 3s 5ms/step - loss: 2.3158 - accuracy: 0.5592 - val_loss: 0.9227 - val_accuracy: 0.7822
Epoch 2/50
760/760 [==============================] - 3s 4ms/step - loss: 0.9019 - accuracy: 0.8216 - val_loss: 0.6761 - val_accuracy: 0.8460
Epoch 3/50
760/760 [==============================] - 3s 4ms/step - loss: 0.5843 - accuracy: 0.8800 - val_loss: 0.4399 - val_accuracy: 0.8992
Epoch 4/50
760/760 [==============================] - 3s 4ms/step - loss: 0.4291 - accuracy: 0.9072 - val_loss: 0.3565 - val_accuracy: 0.9170
Epoch 5/50
760/760 [==============================] - 3s 4ms/step - loss: 0.3446 - accuracy: 0.9261 - val_loss: 0.3044 - val_accuracy: 0.9318
Epoch 6/50
760/760 [==============================] - 3s 4ms/step - loss: 0.2867 - accuracy: 0.9361 - val_loss: 0.3058 - val_accuracy: 0.9296
Epoch 7/50
760/760 [==============================] - 3s 4ms/step - loss: 0.2351 - accuracy: 0.9445 - v









100%|██████████| 9/9 [08:45<00:00, 58.43s/it]


de 0.9639
fr 0.9706
it 0.2337


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
20,CNN1D,mlc_frde,accuracy,NaN,0.6344,0.7696,0.8332,0.8935,0.9219,0.9481,0.9558,0.9481,0.9639
21,CNN1D,mlc_frde,avg_recall,NaN,0.552,0.6918,0.7604,0.8104,0.8495,0.9135,0.9186,0.8914,0.944


### RNN

In [216]:
task = 'mlc_frde'
model = 'RNN'
metrics = ['accuracy','avg_recall']

for m in metrics:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000,10000,15000]):
     run_ml_RNN(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,\
                  X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,\
                  class_weight_dict_de_fr,obs)

df_results[(df_results['model']==model) & (df_results['task']==task)]


















  0%|          | 0/8 [00:00<?, ?it/s]

236 17124 40724
Epoch 1/50
637/637 [==============================] - 8s 12ms/step - loss: 1.1409 - accuracy: 0.7104 - val_loss: 2.1805 - val_accuracy: 0.6185
Epoch 2/50
637/637 [==============================] - 7s 11ms/step - loss: 0.4501 - accuracy: 0.9040 - val_loss: 2.3420 - val_accuracy: 0.6565
Epoch 3/50
637/637 [==============================] - 7s 11ms/step - loss: 0.3229 - accuracy: 0.9309 - val_loss: 2.2371 - val_accuracy: 0.6651
Epoch 4/50
637/637 [==============================] - 7s 11ms/step - loss: 0.2743 - accuracy: 0.9379 - val_loss: 2.3024 - val_accuracy: 0.6742
Epoch 5/50
637/637 [==============================] - 7s 11ms/step - loss: 0.2362 - accuracy: 0.9467 - val_loss: 2.3325 - val_accuracy: 0.6729
Epoch 6/50
637/637 [==============================] - 7s 11ms/step - loss: 0.2320 - accuracy: 0.9441 - val_loss: 2.4781 - val_accuracy: 0.6870

de 0.5207



















 12%|█▎        | 1/8 [00:47<05:30, 47.27s/it]

fr 0.8942
it 0.1253
95 17124 40874
Epoch 1/50
639/639 [==============================] - 8s 12ms/step - loss: 1.4108 - accuracy: 0.6459 - val_loss: 1.4249 - val_accuracy: 0.6997
Epoch 2/50
639/639 [==============================] - 7s 11ms/step - loss: 0.4626 - accuracy: 0.8963 - val_loss: 1.3521 - val_accuracy: 0.7703
Epoch 3/50
639/639 [==============================] - 7s 11ms/step - loss: 0.3354 - accuracy: 0.9269 - val_loss: 1.3029 - val_accuracy: 0.7883
Epoch 4/50
639/639 [==============================] - 7s 11ms/step - loss: 0.2987 - accuracy: 0.9319 - val_loss: 1.2810 - val_accuracy: 0.7990
Epoch 5/50
639/639 [==============================] - 7s 11ms/step - loss: 0.2572 - accuracy: 0.9406 - val_loss: 1.3810 - val_accuracy: 0.7883
Epoch 6/50
639/639 [==============================] - 7s 11ms/step - loss: 0.2102 - accuracy: 0.9505 - val_loss: 1.4201 - val_accuracy: 0.7628
Epoch 7/50
639/639 [==============================] - 7s 11ms/step - loss: 0.2014 - accuracy: 0.9525 - val_


















 25%|██▌       | 2/8 [01:55<05:20, 53.45s/it]

...


















 88%|████████▊ | 7/8 [11:22<01:49, 109.50s/it]

fr 0.945
it 0.3277
2 17124 47124
Epoch 1/50
737/737 [==============================] - 8s 11ms/step - loss: 2.1359 - accuracy: 0.5504 - val_loss: 0.7170 - val_accuracy: 0.8080
Epoch 2/50
737/737 [==============================] - 8s 10ms/step - loss: 1.0413 - accuracy: 0.7902 - val_loss: 0.4941 - val_accuracy: 0.8696
Epoch 3/50
737/737 [==============================] - 8s 10ms/step - loss: 0.6744 - accuracy: 0.8505 - val_loss: 0.4092 - val_accuracy: 0.8849
Epoch 4/50
737/737 [==============================] - 8s 10ms/step - loss: 0.5352 - accuracy: 0.8774 - val_loss: 0.3343 - val_accuracy: 0.9134
Epoch 5/50
737/737 [==============================] - 8s 10ms/step - loss: 0.4539 - accuracy: 0.8964 - val_loss: 0.3315 - val_accuracy: 0.9169
Epoch 6/50
737/737 [==============================] - 8s 10ms/step - loss: 0.3984 - accuracy: 0.9046 - val_loss: 0.2943 - val_accuracy: 0.9281
Epoch 7/50
737/737 [==============================] - 8s 11ms/step - loss: 0.3578 - accuracy: 0.9148 - val_lo


















100%|██████████| 8/8 [15:03<00:00, 112.96s/it]

fr 0.9667
it 0.4723


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
6,RNN,mlc_frde,accuracy,NaN,0.5207,0.7185,0.7984,0.8726,0.911,0.9408,0.9514,0.9614,NaN
7,RNN,mlc_frde,avg_recall,NaN,0.3379,0.5702,0.6576,0.7467,0.8115,0.8679,0.8869,0.9235,NaN
8,RNN,mlc_frde,accuracy,NaN,0.5207,0.7185,0.7984,0.8726,0.911,0.9408,0.9514,0.9614,NaN
9,RNN,mlc_frde,avg_recall,NaN,0.3379,0.5702,0.6576,0.7467,0.8115,0.8679,0.8869,0.9235,NaN


In [ ]:
df_results

# Shop to Shop Transfer

In [128]:
df_rewe = df_de[df_de['shop'] == 'shop.rewe.de']
df_cap= df_de[df_de['shop'] == 'CAP MARKT']
df_rewe2 = df_de[df_de['shop'] == 'REWE']
df_rewe3 = df_de[df_de['shop'] == 'ALDI Nord']
df_rewe4 = df_de[df_de['shop'] == 'ALDI Süd']
df_rewe5 = df_de[df_de['shop'] == 'Treff 3000']
df_rewe6 = df_de[df_de['shop'] == 'Spar']

df_rewe_united = df_rewe.append(df_cap)#.append(df_rewe2).append(df_rewe3).append(df_rewe4).append(df_rewe5).append(df_rewe6)
df_edeka= df_de[df_de['shop'] == 'REWE']
df_real = df_de[df_de['shop'] == 'EDEKA Center']

len(df_rewe),len(df_edeka),len(df_real)

(9124, 2003, 1855)

In [129]:
from sklearn.model_selection import train_test_split
def split_train_abs(df):
    X_train, X_val_test, y_train, y_val_test  = train_test_split(df['text'], df['cc5'], random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test , y_val_test, train_size=.5,random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train_de_rewe, X_val_de_rewe, X_test_de_rewe, y_train_de_rewe, y_val_de_rewe, y_test_de_rewe = split_train_abs(df_rewe)
X_train_de_rewe, X_val_de_rewe2, X_test_de_rewe2, y_train_de_rewe2, y_val_de_rewe2, y_test_de_rewe2 = split_train_abs(df_rewe_united)

X_test_tokens_de = tokenizer_all.texts_to_sequences(X_test_de)

X_train_pad_de = pad_sequences(X_train_tokens_de,maxlen=seq_len, padding='post')



X_train_emb_de_rewe = np.array(list(model_helper.text_to_embed(X_train_de_rewe, ['de' for a in X_train_de_rewe], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_de_rewe = np.array(list(model_helper.text_to_embed(X_val_de_rewe, ['de' for a in X_val_de_rewe], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_de_rewe = np.array(list(model_helper.text_to_embed(X_test_de_rewe, ['de' for a in X_test_de_rewe], de_git_embed, fr_git_embed, seq_len=seq_len)))

X_train_emb_de_rewe2 = np.array(list(model_helper.text_to_embed(X_train_de_rewe2, ['de' for a in X_train_de_rewe2], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_de_rewe2 = np.array(list(model_helper.text_to_embed(X_val_de_rewe2, ['de' for a in X_val_de_rewe2], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_de_rewe2 = np.array(list(model_helper.text_to_embed(X_test_de_rewe2, ['de' for a in X_test_de_rewe2], de_git_embed, fr_git_embed, seq_len=seq_len)))

################################################################

X_emb_edeka = np.array(list(model_helper.text_to_embed(df_edeka['text'], df_edeka['lang'], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_emb_real = np.array(list(model_helper.text_to_embed(df_real['text'], df_real['lang'], de_git_embed, fr_git_embed, seq_len=seq_len)))

y_train_enc_de_rewe = encode_label(y_train_de_rewe)
y_val_enc_de_rewe = encode_label(y_val_de_rewe)
y_test_enc_de_rewe = encode_label(y_test_de_rewe)

y_train_enc_de_rewe2 = encode_label(y_train_de_rewe2)
y_val_enc_de_rewe2 = encode_label(y_val_de_rewe2)
y_test_enc_de_rewe2 = encode_label(y_test_de_rewe2)


###################################################

y_enc_edeka = encode_label(df_edeka['cc5'])
y_enc_real = encode_label(df_real['cc5'])

class_weight_dict_de = get_weigth_dict(y_train_de_rewe)

TypeError: ignored

In [ ]:
dropout_rate=.2
lr = .005
opt = Adam(lr=lr, decay=lr/100)

input_layer = Input(shape = (seq_len,embedding_dim,), name='text_input')

filter_sizes = [1,2,5]
maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv1D(100, kernel_size=filter_sizes[i], kernel_initializer='he_normal', activation='relu')(input_layer)
    maxpool_pool.append(AveragePooling1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

pool_layer = Concatenate(axis=1)(maxpool_pool)   
falt_layer = Flatten(name='flat')(pool_layer)

drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de',kernel_regularizer=l1_l2(l1=5e-6, l2=5e-9))(drop_layer) 

de_rewe = Model(inputs = [input_layer], outputs = [pred_layer])
de_rewe.summary()

de_rewe.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

hist = de_rewe.fit(x = X_train_emb_de_rewe, y = y_train_enc_de_rewe,\
                validation_data = (X_val_emb_de_rewe, y_val_enc_de_rewe), \
                epochs = 50, batch_size = 32, shuffle = True,verbose =0, \
                class_weight = class_weight_dict_de, \
                callbacks = [early_stopping])

In [ ]:
dropout_rate=.2
lr = .005
opt = Adam(lr=lr, decay=lr/100)

input_layer = Input(shape = (seq_len,embedding_dim,), name='text_input')

filter_sizes = [1,2,5]
maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv1D(100, kernel_size=filter_sizes[i], kernel_initializer='he_normal', activation='relu')(input_layer)
    maxpool_pool.append(AveragePooling1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

pool_layer = Concatenate(axis=1)(maxpool_pool)   
falt_layer = Flatten(name='flat')(pool_layer)

drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de',kernel_regularizer=l1_l2(l1=5e-6, l2=5e-9))(drop_layer) 

de_rewe2 = Model(inputs = [input_layer], outputs = [pred_layer])
de_rewe2.summary()

de_rewe2.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

hist = de_rewe.fit(x = X_train_emb_de_rewe2, y = y_train_enc_de_rewe2,\
                validation_data = (X_val_emb_de_rewe2, y_val_enc_de_rewe2), \
                epochs = 50, batch_size = 32, shuffle = True,verbose =0, \
                class_weight = class_weight_dict_de, \
                callbacks = [early_stopping])

In [ ]:
def test_xy(X,y,string,model):
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    print('accuracy %s'% string,accuracy_score(pred,y))
    print('b_accuracy %s'%  string,balanced_accuracy_score(pred, y))

test_xy(X_test_emb_de,y_test_de,'german',de_rewe)
test_xy(X_emb_edeka,df_edeka['cc5'],'edeka',de_rewe)
test_xy(X_emb_real,df_real['cc5'],'real',de_rewe)

test_xy(X_test_emb_de,y_test_de,'german',de_rewe2)
test_xy(X_emb_edeka,df_edeka['cc5'],'edeka',de_rewe2)
test_xy(X_emb_real,df_real['cc5'],'real',de_rewe2)



# Multilingual Transfer

load embedings  
we only load a "slim" version of the embeddings, which are a subset of the vocab (less than 5%) the time loading the embeddings decreases from 15 min to 7 sec and colab is capable of loading more than two languages (embedding)

### Co2

In [ ]:
#!pip install experiment-impact-tracker
#!mkdir logs
#from experiment_impact_tracker.compute_tracker import ImpactTracker
#tracker = ImpactTracker('logs')
#tracker.launch_impact_monitor()
#info = tracker.get_latest_info_and_check_for_errors()
#!ls logs/impacttracker
#!cat logs/impacttracker/impact_tracker_log.log
#!wget 'https://raw.githubusercontent.com/ELehmann91/Thesis_Multilingual_Transferlearning/master/data/model.json'
!create-compute-appendix logs/ --site_spec model.json --output_dir logs/

In [ ]:
task = 'slc_de_zstf'
model = 'LogReg'
metrics = ['accuracy','avg_recall']

for m in metrics:
    df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)
    

tf_idf_log_reg(X_train_de,y_train_de,X_test_fr,y_test_fr,15000)

df_results[(df_results['model']==model) & (df_results['task']==task)]

## CNN 1D

In [141]:
dropout_rate= 0.5
filter_sizes =  [1,2,5]
num_filters = 80
lr = .01
opt = Adam(lr=lr, decay=lr/150)

                      
input_layer = Input(shape = (seq_len,), name='text_input')
embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)
maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
    maxpool_pool.append(AvgPool1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

pool_layer = Concatenate(axis=1)(maxpool_pool)  
falt_layer = Flatten(name='flat')(pool_layer)
#dense_layer = Dense(150,activation='relu')(falt_layer)

drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)
print(X_train_pad_de.shape, y_train_enc_de.shape)
print(de_cnn1d.summary())
hist = de_cnn1d.fit(x = X_train_pad_de, y = y_train_enc_de,\
                validation_data = (X_val_pad_de, y_val_enc_de), \
                epochs = 150, batch_size = 64, shuffle = True, class_weight = class_weight_dict_de,  \
                callbacks = [early_stopping])
#verbose =0,


all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)


(23597, 31) (23597, 75)
Model: "model_181"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_149 (Embedding)       (None, 31, 300)      6590700     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_341 (Conv1D)             (None, 31, 80)       24080       embedding_149[0][0]              
__________________________________________________________________________________________________
conv1d_342 (Conv1D)             (None, 30, 80)       48080       embedding_149[0][0]              
__________________________________________________________________

### deeper prediction analysis

Prediction / Certainty Treshold

In [ ]:
def test_xy(X,y,string,model,z,ret=False):
    obs = len(X)
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    y_pred_max = y_pred.max(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    df_pred = pd.DataFrame(zip(X,pred,y,y_pred_max),columns=['embed','pred','y','y_pred_max'])
    df_pred = df_pred[df_pred['y_pred_max']>z]
    print('accuracy %s'% string,accuracy_score(df_pred['pred'],df_pred['y']))
    print('b_accuracy %s'%  string,balanced_accuracy_score(df_pred['pred'],df_pred['y']))
    print(len(df_pred),'of',len(X),int(len(df_pred)/len(X)*100),'%',len(df_pred.pred.unique()))
    if ret:
        return df_pred
test_xy(X_test_emb_de,y_test_de,'german',de_avg_pool,.65)
test_xy(X_traprint(classification_report(y,pred))in_emb_fr,y_train_fr,'french',de_avg_pool,.8)
    #return pred, y

In [ ]:
print(classification_report(y,pred))

Accuracy for higher hirachies

In [ ]:
with open('coicop_5_4.txt') as json_file:#
    coicop_5_4 = json.load(json_file)

with open('coicop_5_3.txt') as json_file:#
    coicop_5_3 = json.load(json_file)

y_pred = de_avg_pool.predict([X_test_emb_fr])
y_pred_arg = y_pred.argmax(axis=1)
pred= [encoder.classes_[y] for y in y_pred_arg]

y_pr_lab3 = [coicop_5_3[cc5] for cc5 in pred]
y_pr_lab4 = [coicop_5_4[cc5] for cc5 in pred]
y_lab3 = [coicop_5_3[cc5] for cc5 in y_test_fr]
y_lab4 = [coicop_5_4[cc5] for cc5 in y_test_fr]
print('accuracy %s'% accuracy_score(y_pr_lab3,y_lab3))
print('b_accuracy %s'%  balanced_accuracy_score(y_pr_lab3, y_lab3))
print('accuracy %s'% accuracy_score(y_pr_lab4,y_lab4))
print('b_accuracy %s'%  balanced_accuracy_score(y_pr_lab4, y_lab4))
#print(classification_report(y_lab4,y_pr_lab4))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

y_pred = de_avg_pool.predict([X_test_emb_fr])
y_pred_arg = y_pred.argmax(axis=1)
pred= [encoder.classes_[y] for y in y_pred_arg]

y_pr_lab4 = [coicop_5_4[cc5] for cc5 in pred]
y_lab4 = [coicop_5_4[cc5] for cc5 in y_test_fr]

label = pd.Series(y_lab4).unique()
label.sort()
cm = confusion_matrix(y_lab4,y_pr_lab4,labels=label)#, normalize='true')

df_cm = pd.DataFrame(cm, label,label)
plt.figure(figsize=(15,15))
sn.set(font_scale=.7) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 9}) # font size

plt.show()

### Transferlearning

In [142]:
#test_xy(X_test_emb_de,y_test_de,'german',model_new)
#test_xy(X_test_emb_fr,y_test_fr,'french',model_new)

In [196]:
dropout_rate= 0.55
filter_sizes =  [1,2,5]
num_filters = 85
lr = .01
opt = Adam(lr=lr, decay=lr/150)

                      
input_layer = Input(shape = (seq_len,), name='text_input')
embedd_seq = Embedding(vocab_size_all, embedding_dim, weights = [embedding_matrix_all], input_length = seq_len, trainable = False, mask_zero=False)(input_layer)
maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv1D(num_filters, kernel_size=filter_sizes[i],  activation='relu')(embedd_seq) #kernel_initializer='he_normal',
    maxpool_pool.append(AvgPool1D(pool_size = seq_len-filter_sizes[i]+1)(conv))

pool_layer = Concatenate(axis=1)(maxpool_pool)  
falt_layer = Flatten(name='flat')(pool_layer)
#dense_layer = Dense(150,activation='relu')(falt_layer)

drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

de_cnn1d = Model(inputs = [input_layer], outputs = pred_layer)
de_cnn1d.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)
print(X_train_pad_de.shape, y_train_enc_de.shape)
print(de_cnn1d.summary())
hist = de_cnn1d.fit(x = X_train_pad_de, y = y_train_enc_de,\
                validation_data = (X_val_pad_de, y_val_enc_de), \
                epochs = 150, batch_size = 64, shuffle = True, class_weight = class_weight_dict_de,  \
                callbacks = [early_stopping])
#verbose =0,

de_cnn1d_tf.save_weights('de_cnn1d_tf')
all_tests(de_cnn1d,X_test_pad_de,X_test_pad_fr,X_pad_it)


(23597, 31) (23597, 75)
Model: "model_225"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_151 (Embedding)       (None, 31, 300)      6846600     text_input[0][0]                 
__________________________________________________________________________________________________
conv1d_347 (Conv1D)             (None, 31, 85)       25585       embedding_151[0][0]              
__________________________________________________________________________________________________
conv1d_348 (Conv1D)             (None, 30, 85)       51085       embedding_151[0][0]              
__________________________________________________________________

In [210]:
def transfer_cnn(few_shot,freeze,lr):

    de_cnn1d_tf.load_weights('de_cnn1d_tf')

    opt = Adam(lr=lr, decay=lr/100)
    inp = de_cnn1d_tf.input
    out = de_cnn1d_tf.output #get_layer('output_de').output

    # create a new network between inp and out
    de_cnn1d_transfer = Model(inp, out)    

    de_cnn1d_transfer.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    #freeze
    for l , layer in enumerate(de_cnn1d_transfer.layers[:-2]):
        de_cnn1d_transfer.layers[l].trainable = (freeze==False)

    idx = np.random.randint(len(X_train_pad_fr), size=few_shot)

    de_cnn1d_transfer.fit(x = X_train_pad_fr[idx,:], y = y_train_enc_fr[idx,:],\
                    validation_data = (X_val_pad_fr, y_val_enc_fr), \
                    epochs = 150, batch_size = 8, shuffle = True, \
                    class_weight = class_weight_dict_fr, \
                    callbacks = [early_stopping])
    print(few_shot)
    print(de_cnn1d_transfer.summary())
    #all_tests(de_cnn1d_transfer,X_test_pad_de,X_test_pad_fr,X_pad_it)

    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test_fr = de_cnn1d_transfer.predict(X_test_pad_fr)
    y_pred_arg_fr = y_pred_test_fr.argmax(axis=1)
    y_pred_test_fr = [encoder.classes_[y] for y in y_pred_arg_fr]
    
    df_results[str(few_shot)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='accuracy')] = round(accuracy_score(y_pred_test_fr, y_test_fr),4)
    df_results[str(few_shot)][(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']=='avg_recall')] = round(balanced_accuracy_score(y_pred_test_fr, y_test_fr),4)

In [217]:
 task = 'slc_de_tf'
model = 'CNN1D_fix_.01'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000]):
     transfer_cnn(obs,True,lr = .01)

df_results[(df_results['model']==model) & (df_results['task']==task)]


















  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/150
13/13 [==============================] - 1s 112ms/step - loss: 58.4846 - accuracy: 0.4100 - val_loss: 20.0345 - val_accuracy: 0.4247
Epoch 2/150
13/13 [==============================] - 1s 104ms/step - loss: 7.4501 - accuracy: 0.6700 - val_loss: 17.4578 - val_accuracy: 0.4874
Epoch 3/150
13/13 [==============================] - 1s 103ms/step - loss: 2.7151 - accuracy: 0.8400 - val_loss: 16.4788 - val_accuracy: 0.5263
Epoch 4/150
13/13 [==============================] - 1s 104ms/step - loss: 0.1801 - accuracy: 0.9400 - val_loss: 16.0109 - val_accuracy: 0.5561
Epoch 5/150
13/13 [==============================] - 1s 103ms/step - loss: 0.0781 - accuracy: 0.9900 - val_loss: 15.8580 - val_accuracy: 0.5648
Epoch 6/150
13/13 [==============================] - 1s 104ms/step - loss: 0.0556 - accuracy: 0.9900 - val_loss: 15.8206 - val_accuracy: 0.5683
Epoch 7/150
13/13 [==============================] - 1s 102ms/step - loss: 0.1396 - accuracy: 0.9700 - val_loss: 15.9386 - val_accuracy


















 17%|█▋        | 1/6 [00:16<01:21, 16.31s/it]

Epoch 1/150
32/32 [==============================] - 1s 34ms/step - loss: 44.2542 - accuracy: 0.3480 - val_loss: 23.0732 - val_accuracy: 0.3676
Epoch 2/150
32/32 [==============================] - 1s 30ms/step - loss: 35.5201 - accuracy: 0.3680 - val_loss: 22.5240 - val_accuracy: 0.3553
Epoch 3/150
32/32 [==============================] - 1s 30ms/step - loss: 33.3145 - accuracy: 0.3480 - val_loss: 22.2261 - val_accuracy: 0.3577
Epoch 4/150
32/32 [==============================] - 1s 30ms/step - loss: 26.4459 - accuracy: 0.3560 - val_loss: 22.0339 - val_accuracy: 0.3570
Epoch 5/150
32/32 [==============================] - 1s 31ms/step - loss: 21.8579 - accuracy: 0.3680 - val_loss: 21.5832 - val_accuracy: 0.3690
Epoch 6/150
32/32 [==============================] - 1s 30ms/step - loss: 21.8432 - accuracy: 0.3920 - val_loss: 21.3286 - val_accuracy: 0.3676
Epoch 7/150
32/32 [==============================] - 1s 30ms/step - loss: 18.4161 - accuracy: 0.3720 - val_loss: 21.3367 - val_accuracy:


















 33%|███▎      | 2/6 [00:57<01:34, 23.73s/it]

...

Epoch 1/150
250/250 [==============================] - 2s 6ms/step - loss: 47.7234 - accuracy: 0.3145 - val_loss: 19.0959 - val_accuracy: 0.3770
Epoch 2/150
250/250 [==============================] - 1s 6ms/step - loss: 34.5191 - accuracy: 0.3350 - val_loss: 15.8548 - val_accuracy: 0.4064
Epoch 3/150
250/250 [==============================] - 1s 6ms/step - loss: 24.5476 - accuracy: 0.4100 - val_loss: 12.9168 - val_accuracy: 0.4632
Epoch 4/150
250/250 [==============================] - 1s 6ms/step - loss: 18.9539 - accuracy: 0.4480 - val_loss: 11.1191 - val_accuracy: 0.4916
Epoch 5/150
250/250 [==============================] - 1s 6ms/step - loss: 14.9280 - accuracy: 0.5015 - val_loss: 8.9398 - val_accuracy: 0.5494
Epoch 6/150
250/250 [==============================] - 1s 6ms/step - loss: 12.5475 - accuracy: 0.5355 - val_loss: 7.8405 - val_accuracy: 0.5757
Epoch 7/150
250/250 [==============================] - 1s 6ms/step - loss: 9.3374 - accuracy: 0.5625 - val_loss: 6.9541 - val_accura


















 83%|████████▎ | 5/6 [03:25<00:39, 39.72s/it]

Epoch 1/150
625/625 [==============================] - 2s 4ms/step - loss: 39.7470 - accuracy: 0.3370 - val_loss: 15.0558 - val_accuracy: 0.4334
Epoch 2/150
625/625 [==============================] - 2s 4ms/step - loss: 20.4883 - accuracy: 0.4378 - val_loss: 9.0915 - val_accuracy: 0.5242
Epoch 3/150
625/625 [==============================] - 2s 4ms/step - loss: 13.3867 - accuracy: 0.5134 - val_loss: 6.5894 - val_accuracy: 0.5971
Epoch 4/150
625/625 [==============================] - 2s 4ms/step - loss: 10.2249 - accuracy: 0.5826 - val_loss: 5.3714 - val_accuracy: 0.6573
Epoch 5/150
625/625 [==============================] - 2s 4ms/step - loss: 8.0204 - accuracy: 0.6220 - val_loss: 4.5805 - val_accuracy: 0.6801
Epoch 6/150
625/625 [==============================] - 2s 4ms/step - loss: 5.9039 - accuracy: 0.6564 - val_loss: 5.1434 - val_accuracy: 0.6678
Epoch 7/150
625/625 [==============================] - 2s 4ms/step - loss: 5.2803 - accuracy: 0.6760 - val_loss: 4.0664 - val_accuracy: 0


















100%|██████████| 6/6 [05:15<00:00, 52.61s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
10,CNN1D_fix_.01,slc_de_tf,accuracy,NaN,0.5489,0.5271,0.6606,0.6932,0.7629,0.828,NaN,NaN,NaN
11,CNN1D_fix_.01,slc_de_tf,avg_recall,NaN,0.2333,0.2737,0.415,0.458,0.5663,0.6466,NaN,NaN,NaN


In [218]:
 task = 'slc_de_tf'
model = 'CNN1D_free_.1'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000]):
     transfer_cnn(obs,False,lr = .1)

df_results[(df_results['model']==model) & (df_results['task']==task)]


















  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/150
13/13 [==============================] - 1s 81ms/step - loss: 58.7097 - accuracy: 0.2800 - val_loss: 30.9634 - val_accuracy: 0.3388
Epoch 2/150
13/13 [==============================] - 1s 73ms/step - loss: 38.9764 - accuracy: 0.4000 - val_loss: 34.8204 - val_accuracy: 0.3203
Epoch 3/150
13/13 [==============================] - 1s 71ms/step - loss: 18.9396 - accuracy: 0.4200 - val_loss: 34.4400 - val_accuracy: 0.3581
Epoch 4/150
13/13 [==============================] - 1s 72ms/step - loss: 11.4713 - accuracy: 0.5000 - val_loss: 30.0681 - val_accuracy: 0.3826
Epoch 5/150
13/13 [==============================] - 1s 72ms/step - loss: 4.9668 - accuracy: 0.6100 - val_loss: 29.5258 - val_accuracy: 0.4093
Epoch 6/150
13/13 [==============================] - 1s 70ms/step - loss: 2.4351 - accuracy: 0.6700 - val_loss: 30.7599 - val_accuracy: 0.4208
Epoch 7/150
13/13 [==============================] - 1s 70ms/step - loss: 0.8934 - accuracy: 0.8400 - val_loss: 29.1916 - val_accuracy: 0.


















 17%|█▋        | 1/6 [00:15<01:18, 15.76s/it]

Epoch 1/150
32/32 [==============================] - 2s 67ms/step - loss: 82.1948 - accuracy: 0.4040 - val_loss: 28.6718 - val_accuracy: 0.5112
Epoch 2/150
32/32 [==============================] - 2s 64ms/step - loss: 29.3587 - accuracy: 0.7160 - val_loss: 44.3483 - val_accuracy: 0.5056
Epoch 3/150
32/32 [==============================] - 2s 64ms/step - loss: 9.6074 - accuracy: 0.8240 - val_loss: 44.4396 - val_accuracy: 0.6324
Epoch 4/150
32/32 [==============================] - 2s 64ms/step - loss: 10.7354 - accuracy: 0.8480 - val_loss: 64.2115 - val_accuracy: 0.6310
Epoch 5/150
32/32 [==============================] - 2s 64ms/step - loss: 9.5636 - accuracy: 0.8920 - val_loss: 66.5421 - val_accuracy: 0.6384
Epoch 6/150
32/32 [==============================] - 3s 97ms/step - loss: 45.3355 - accuracy: 0.9040 - val_loss: 67.4917 - val_accuracy: 0.6833
250
Model: "model_283"
__________________________________________________________________________________________________
Layer (type)    


















 33%|███▎      | 2/6 [00:30<01:01, 15.41s/it]

...

Epoch 1/150
250/250 [==============================] - 9s 38ms/step - loss: 53.2798 - accuracy: 0.5325 - val_loss: 51.6169 - val_accuracy: 0.5410
Epoch 2/150
250/250 [==============================] - 9s 37ms/step - loss: 51.1401 - accuracy: 0.5805 - val_loss: 33.1383 - val_accuracy: 0.6132
Epoch 3/150
250/250 [==============================] - 9s 37ms/step - loss: 37.2679 - accuracy: 0.6475 - val_loss: 34.2666 - val_accuracy: 0.6440
Epoch 4/150
250/250 [==============================] - 9s 37ms/step - loss: 20.6467 - accuracy: 0.7020 - val_loss: 37.7952 - val_accuracy: 0.6367
Epoch 5/150
250/250 [==============================] - 9s 37ms/step - loss: 15.7559 - accuracy: 0.7130 - val_loss: 35.1998 - val_accuracy: 0.6556
Epoch 6/150
250/250 [==============================] - 9s 37ms/step - loss: 8.0515 - accuracy: 0.7490 - val_loss: 39.5711 - val_accuracy: 0.6790
Epoch 7/150
250/250 [==============================] - 9s 37ms/step - loss: 8.7051 - accuracy: 0.7565 - val_loss: 36.4712 - v


















 83%|████████▎ | 5/6 [02:28<00:34, 34.85s/it]

Epoch 1/150
625/625 [==============================] - 22s 36ms/step - loss: 66.9284 - accuracy: 0.5390 - val_loss: 30.1220 - val_accuracy: 0.6223
Epoch 2/150
625/625 [==============================] - 22s 35ms/step - loss: 57.8626 - accuracy: 0.6014 - val_loss: 23.9484 - val_accuracy: 0.7158
Epoch 3/150
625/625 [==============================] - 22s 35ms/step - loss: 17.2127 - accuracy: 0.6670 - val_loss: 22.9610 - val_accuracy: 0.6202
Epoch 4/150
625/625 [==============================] - 22s 35ms/step - loss: 7.1448 - accuracy: 0.6708 - val_loss: 23.2507 - val_accuracy: 0.6896
Epoch 5/150
625/625 [==============================] - 22s 35ms/step - loss: 4.2274 - accuracy: 0.7000 - val_loss: 21.6942 - val_accuracy: 0.7001
Epoch 6/150
625/625 [==============================] - 22s 35ms/step - loss: 3.8591 - accuracy: 0.7302 - val_loss: 20.6299 - val_accuracy: 0.7228
Epoch 7/150
625/625 [==============================] - 22s 35ms/step - loss: 3.9585 - accuracy: 0.7194 - val_loss: 21.743


















100%|██████████| 6/6 [06:33<00:00, 65.52s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,CNN1D_free_.1,slc_de_tf,accuracy,NaN,0.4862,0.6785,0.7187,0.7744,0.7061,0.7205,NaN,NaN,NaN
1,CNN1D_free_.1,slc_de_tf,avg_recall,NaN,0.3366,0.5133,0.5497,0.5252,0.509,0.5631,NaN,NaN,NaN


In [212]:
 task = 'slc_de_tf'
model = 'CNN1D_free_.01'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000]):
     transfer_cnn(obs,False,lr = .01)

df_results[(df_results['model']==model) & (df_results['task']==task)]


















  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/150
13/13 [==============================] - 1s 111ms/step - loss: 33.5067 - accuracy: 0.2600 - val_loss: 19.4199 - val_accuracy: 0.4334
Epoch 2/150
13/13 [==============================] - 1s 99ms/step - loss: 6.4874 - accuracy: 0.6200 - val_loss: 16.9841 - val_accuracy: 0.5007
Epoch 3/150
13/13 [==============================] - 1s 100ms/step - loss: 1.2603 - accuracy: 0.8400 - val_loss: 16.3206 - val_accuracy: 0.5399
Epoch 4/150
13/13 [==============================] - 1s 100ms/step - loss: 0.5557 - accuracy: 0.9100 - val_loss: 16.1716 - val_accuracy: 0.5557
Epoch 5/150
13/13 [==============================] - 1s 100ms/step - loss: 0.0723 - accuracy: 0.9700 - val_loss: 16.0269 - val_accuracy: 0.5676
Epoch 6/150
13/13 [==============================] - 1s 99ms/step - loss: 0.1024 - accuracy: 0.9900 - val_loss: 16.0012 - val_accuracy: 0.5704
Epoch 7/150
13/13 [==============================] - 1s 102ms/step - loss: 0.0550 - accuracy: 0.9900 - val_loss: 16.0578 - val_accuracy: 


















 17%|█▋        | 1/6 [00:24<02:01, 24.32s/it]

Epoch 1/150
32/32 [==============================] - 2s 65ms/step - loss: 30.9843 - accuracy: 0.4200 - val_loss: 13.9306 - val_accuracy: 0.5277
Epoch 2/150
32/32 [==============================] - 2s 62ms/step - loss: 6.0297 - accuracy: 0.8040 - val_loss: 11.1020 - val_accuracy: 0.6237
Epoch 3/150
32/32 [==============================] - 2s 62ms/step - loss: 0.3034 - accuracy: 0.9360 - val_loss: 10.7671 - val_accuracy: 0.6468
Epoch 4/150
32/32 [==============================] - 2s 62ms/step - loss: 0.1261 - accuracy: 0.9640 - val_loss: 10.6679 - val_accuracy: 0.6587
Epoch 5/150
32/32 [==============================] - 2s 63ms/step - loss: 0.0341 - accuracy: 0.9880 - val_loss: 10.5155 - val_accuracy: 0.6647
Epoch 6/150
32/32 [==============================] - 2s 62ms/step - loss: 0.1611 - accuracy: 0.9680 - val_loss: 10.4977 - val_accuracy: 0.6629
Epoch 7/150
32/32 [==============================] - 2s 62ms/step - loss: 0.0676 - accuracy: 0.9880 - val_loss: 10.5276 - val_accuracy: 0.663


















 33%|███▎      | 2/6 [00:55<01:45, 26.42s/it]

...

Epoch 1/150
250/250 [==============================] - 10s 39ms/step - loss: 20.5453 - accuracy: 0.6255 - val_loss: 3.6497 - val_accuracy: 0.7961
Epoch 2/150
250/250 [==============================] - 10s 38ms/step - loss: 2.6502 - accuracy: 0.8860 - val_loss: 2.9030 - val_accuracy: 0.8189
Epoch 3/150
250/250 [==============================] - 10s 38ms/step - loss: 0.8636 - accuracy: 0.9335 - val_loss: 2.7038 - val_accuracy: 0.8469
Epoch 4/150
250/250 [==============================] - 10s 38ms/step - loss: 0.5282 - accuracy: 0.9690 - val_loss: 2.8766 - val_accuracy: 0.8591
Epoch 5/150
250/250 [==============================] - 10s 39ms/step - loss: 0.3056 - accuracy: 0.9700 - val_loss: 2.8993 - val_accuracy: 0.8542
Epoch 6/150
250/250 [==============================] - 10s 39ms/step - loss: 0.2789 - accuracy: 0.9765 - val_loss: 3.0182 - val_accuracy: 0.8560
Epoch 7/150
250/250 [==============================] - 9s 38ms/step - loss: 0.1684 - accuracy: 0.9870 - val_loss: 3.2781 - val_ac


















 83%|████████▎ | 5/6 [03:26<00:45, 45.96s/it]

Epoch 1/150
625/625 [==============================] - 23s 36ms/step - loss: 11.8328 - accuracy: 0.7396 - val_loss: 2.1699 - val_accuracy: 0.8718
Epoch 2/150
625/625 [==============================] - 23s 36ms/step - loss: 1.4605 - accuracy: 0.9230 - val_loss: 1.8635 - val_accuracy: 0.8973
Epoch 3/150
625/625 [==============================] - 23s 36ms/step - loss: 0.5634 - accuracy: 0.9506 - val_loss: 2.1407 - val_accuracy: 0.8963
Epoch 4/150
625/625 [==============================] - 23s 36ms/step - loss: 0.3694 - accuracy: 0.9652 - val_loss: 2.3973 - val_accuracy: 0.8977
Epoch 5/150
625/625 [==============================] - 23s 36ms/step - loss: 0.2353 - accuracy: 0.9750 - val_loss: 2.4295 - val_accuracy: 0.9145
Epoch 6/150
625/625 [==============================] - 23s 37ms/step - loss: 0.2856 - accuracy: 0.9770 - val_loss: 2.7956 - val_accuracy: 0.9103
Epoch 7/150
625/625 [==============================] - 23s 36ms/step - loss: 0.2600 - accuracy: 0.9788 - val_loss: 2.8132 - val_a


















100%|██████████| 6/6 [06:06<00:00, 61.17s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
2,CNN1D_free_.01,slc_de_tf,accuracy,NaN,0.5653,0.6606,0.7117,0.779,0.8469,0.8939,NaN,NaN,NaN
3,CNN1D_free_.01,slc_de_tf,avg_recall,NaN,0.2673,0.3676,0.4479,0.5756,0.6409,0.7303,NaN,NaN,NaN


In [213]:
 task = 'slc_de_tf'
model = 'CNN1D_free_.001'
metrics = ['accuracy','avg_recall']

for m in metrics:
    if len(df_results[(df_results['model']==model) & (df_results['task']==task)& (df_results['metric']==m)])==0:
        df_results = df_results.append({'model': model,'task':task,'metric':m}, ignore_index=True)

for obs in tqdm([100,250,500,1000,2000,5000]):
     transfer_cnn(obs,False,lr = .001)

df_results[(df_results['model']==model) & (df_results['task']==task)]


















  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/150
13/13 [==============================] - 1s 110ms/step - loss: 72.7377 - accuracy: 0.3300 - val_loss: 24.0922 - val_accuracy: 0.3644
Epoch 2/150
13/13 [==============================] - 1s 100ms/step - loss: 60.9595 - accuracy: 0.3200 - val_loss: 23.1753 - val_accuracy: 0.3707
Epoch 3/150
13/13 [==============================] - 1s 102ms/step - loss: 52.7246 - accuracy: 0.3300 - val_loss: 22.3902 - val_accuracy: 0.3791
Epoch 4/150
13/13 [==============================] - 1s 100ms/step - loss: 44.0750 - accuracy: 0.3700 - val_loss: 21.6803 - val_accuracy: 0.3840
Epoch 5/150
13/13 [==============================] - 1s 101ms/step - loss: 31.0680 - accuracy: 0.4000 - val_loss: 21.0199 - val_accuracy: 0.3935
Epoch 6/150
13/13 [==============================] - 1s 102ms/step - loss: 29.8502 - accuracy: 0.4300 - val_loss: 20.4812 - val_accuracy: 0.3980
Epoch 7/150
13/13 [==============================] - 1s 100ms/step - loss: 22.1960 - accuracy: 0.4200 - val_loss: 19.9716 - val_ac


















 17%|█▋        | 1/6 [02:48<14:01, 168.37s/it]

Epoch 1/150
32/32 [==============================] - 2s 65ms/step - loss: 39.6539 - accuracy: 0.3320 - val_loss: 22.9583 - val_accuracy: 0.3735
Epoch 2/150
32/32 [==============================] - 2s 62ms/step - loss: 29.5172 - accuracy: 0.3760 - val_loss: 21.2120 - val_accuracy: 0.3882
Epoch 3/150
32/32 [==============================] - 2s 62ms/step - loss: 22.7916 - accuracy: 0.4120 - val_loss: 19.7011 - val_accuracy: 0.4001
Epoch 4/150
32/32 [==============================] - 2s 62ms/step - loss: 18.3346 - accuracy: 0.4800 - val_loss: 18.5753 - val_accuracy: 0.4156
Epoch 5/150
32/32 [==============================] - 2s 62ms/step - loss: 11.7863 - accuracy: 0.5600 - val_loss: 17.6988 - val_accuracy: 0.4233
Epoch 6/150
32/32 [==============================] - 2s 62ms/step - loss: 11.1582 - accuracy: 0.5640 - val_loss: 16.8931 - val_accuracy: 0.4310
Epoch 7/150
32/32 [==============================] - 2s 62ms/step - loss: 7.1356 - accuracy: 0.6280 - val_loss: 16.1623 - val_accuracy: 


















 33%|███▎      | 2/6 [06:45<12:35, 188.99s/it]

...

Epoch 1/150
250/250 [==============================] - 10s 38ms/step - loss: 41.9551 - accuracy: 0.3900 - val_loss: 13.2659 - val_accuracy: 0.4674
Epoch 2/150
250/250 [==============================] - 9s 37ms/step - loss: 18.5222 - accuracy: 0.5445 - val_loss: 8.1255 - val_accuracy: 0.5760
Epoch 3/150
250/250 [==============================] - 9s 38ms/step - loss: 8.9264 - accuracy: 0.6630 - val_loss: 5.9779 - val_accuracy: 0.6706
Epoch 4/150
250/250 [==============================] - 9s 37ms/step - loss: 4.7368 - accuracy: 0.7670 - val_loss: 4.9826 - val_accuracy: 0.7204
Epoch 5/150
250/250 [==============================] - 9s 38ms/step - loss: 2.5291 - accuracy: 0.8340 - val_loss: 4.4183 - val_accuracy: 0.7474
Epoch 6/150
250/250 [==============================] - 9s 37ms/step - loss: 1.2988 - accuracy: 0.8655 - val_loss: 4.0711 - val_accuracy: 0.7694
Epoch 7/150
250/250 [==============================] - 9s 38ms/step - loss: 0.9668 - accuracy: 0.8890 - val_loss: 3.8387 - val_accur


















 83%|████████▎ | 5/6 [18:46<03:56, 236.02s/it]

Epoch 1/150
625/625 [==============================] - 22s 36ms/step - loss: 28.7153 - accuracy: 0.4686 - val_loss: 6.2617 - val_accuracy: 0.6258
Epoch 2/150
625/625 [==============================] - 22s 36ms/step - loss: 6.8705 - accuracy: 0.7102 - val_loss: 3.2581 - val_accuracy: 0.7835
Epoch 3/150
625/625 [==============================] - 22s 36ms/step - loss: 2.4139 - accuracy: 0.8226 - val_loss: 2.4861 - val_accuracy: 0.8290
Epoch 4/150
625/625 [==============================] - 22s 35ms/step - loss: 1.0984 - accuracy: 0.8948 - val_loss: 2.1545 - val_accuracy: 0.8553
Epoch 5/150
625/625 [==============================] - 22s 36ms/step - loss: 0.6231 - accuracy: 0.9192 - val_loss: 1.9535 - val_accuracy: 0.8672
Epoch 6/150
625/625 [==============================] - 23s 37ms/step - loss: 0.3798 - accuracy: 0.9390 - val_loss: 1.8088 - val_accuracy: 0.8777
Epoch 7/150
625/625 [==============================] - 22s 36ms/step - loss: 0.3152 - accuracy: 0.9544 - val_loss: 1.7791 - val_a


















100%|██████████| 6/6 [25:50<00:00, 258.50s/it]


,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
4,CNN1D_free_.001,slc_de_tf,accuracy,NaN,0.524,0.6186,0.6911,0.751,0.8364,0.8963,NaN,NaN,NaN
5,CNN1D_free_.001,slc_de_tf,avg_recall,NaN,0.2208,0.2711,0.4004,0.5132,0.6282,0.7385,NaN,NaN,NaN


In [214]:
df_results

,model,task,metric,0,100,250,500,1000,2000,5000,10000,15000,25000
0,CNN1D_free_.1,slc_de_tf,accuracy,NaN,0.5629,0.6553,0.6834,0.7958,0.8158,0.7401,NaN,NaN,NaN
1,CNN1D_free_.1,slc_de_tf,avg_recall,NaN,0.5234,0.5155,0.5551,0.5742,0.5663,0.6032,NaN,NaN,NaN
2,CNN1D_free_.01,slc_de_tf,accuracy,NaN,0.5653,0.6606,0.7117,0.779,0.8469,0.8939,NaN,NaN,NaN
3,CNN1D_free_.01,slc_de_tf,avg_recall,NaN,0.2673,0.3676,0.4479,0.5756,0.6409,0.7303,NaN,NaN,NaN
4,CNN1D_free_.001,slc_de_tf,accuracy,NaN,0.524,0.6186,0.6911,0.751,0.8364,0.8963,NaN,NaN,NaN
5,CNN1D_free_.001,slc_de_tf,avg_recall,NaN,0.2208,0.2711,0.4004,0.5132,0.6282,0.7385,NaN,NaN,NaN
